# Chapter 10: Inheritance and Polymorphism

## Introduction to Object-Oriented Programming in SystemVerilog

SystemVerilog supports object-oriented programming (OOP) concepts that enable code reusability, modularity, and better organization. This chapter explores inheritance and polymorphism, two fundamental OOP concepts that allow you to create hierarchical relationships between classes and write more flexible, maintainable code.

## Class Inheritance (extends)

Inheritance allows you to create new classes based on existing classes, inheriting their properties and methods while adding new functionality or modifying existing behavior.

### Basic Inheritance Syntax

```systemverilog
class BaseClass;
    // Base class properties and methods
endclass

class DerivedClass extends BaseClass;
    // Derived class inherits from BaseClass
    // Additional properties and methods
endclass
```

### Practical Example: Vehicle Hierarchy

```systemverilog
// Base Vehicle class
class Vehicle;
    string make;
    string model;
    int year;
    
    function new(string mk = "Unknown", string md = "Unknown", int yr = 2024);
        make = mk;
        model = md;
        year = yr;
    endfunction
    
    virtual function void display_info();
        $display("Vehicle: %s %s (%0d)", make, model, year);
    endfunction
    
    virtual function void start_engine();
        $display("Starting engine...");
    endfunction
endclass

// Car class inherits from Vehicle
class Car extends Vehicle;
    int num_doors;
    string fuel_type;
    
    function new(string mk = "Unknown", string md = "Unknown", 
                 int yr = 2024, int doors = 4, string fuel = "Gasoline");
        super.new(mk, md, yr);  // Call parent constructor
        num_doors = doors;
        fuel_type = fuel;
    endfunction
    
    // Override parent method
    virtual function void display_info();
        super.display_info();  // Call parent method
        $display("  Type: Car, Doors: %0d, Fuel: %s", num_doors, fuel_type);
    endfunction
    
    function void open_trunk();
        $display("Opening car trunk...");
    endfunction
endclass

// Motorcycle class inherits from Vehicle
class Motorcycle extends Vehicle;
    bit has_sidecar;
    int engine_cc;
    
    function new(string mk = "Unknown", string md = "Unknown", 
                 int yr = 2024, bit sidecar = 0, int cc = 250);
        super.new(mk, md, yr);
        has_sidecar = sidecar;
        engine_cc = cc;
    endfunction
    
    virtual function void display_info();
        super.display_info();
        $display("  Type: Motorcycle, Engine: %0dcc, Sidecar: %s", 
                 engine_cc, has_sidecar ? "Yes" : "No");
    endfunction
    
    virtual function void start_engine();
        $display("Kick-starting motorcycle engine...");
    endfunction
endclass
```

### **Example 1:** Basic Pet Hierarchy
**Description:** Simple Animal base class with Dog and Cat derived classes. Shows basic inheritance syntax with name and age properties.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_1__basic_pet_hierarchy/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// pet_hierarchy_design.sv
// Basic Pet Hierarchy Example - Chapter 10 Example 1

// Base Animal class
class Animal;
  string name;
  int age;
  
  // Constructor
  function new(string pet_name = "Unknown", int pet_age = 0);
    this.name = pet_name;
    this.age = pet_age;
  endfunction
  
  // Virtual method for making sound (to be overridden)
  virtual function string make_sound();
    return "Some generic animal sound";
  endfunction
  
  // Method to display pet information
  function void display_info();
    $display("Pet Name: %s, Age: %d years", name, age);
  endfunction
  
  // Method to get pet description
  virtual function string get_description();
    return $sformatf("%s is %d years old", name, age);
  endfunction
endclass

// Dog class inheriting from Animal
class Dog extends Animal;
  string breed;
  
  // Constructor
  function new(string pet_name = "Buddy", int pet_age = 0, 
               string dog_breed = "Mixed");
    super.new(pet_name, pet_age);  // Call parent constructor
    this.breed = dog_breed;
  endfunction
  
  // Override make_sound method
  function string make_sound();
    return "Woof! Woof!";
  endfunction
  
  // Override get_description method
  function string get_description();
    return $sformatf("%s is a %d-year-old %s", name, age, breed);
  endfunction
  
  // Dog-specific method
  function void wag_tail();
    $display("%s is wagging tail happily!", name);
  endfunction
endclass

// Cat class inheriting from Animal
class Cat extends Animal;
  bit is_indoor;
  
  // Constructor  
  function new(string pet_name = "Whiskers", int pet_age = 0, 
               bit indoor = 1);
    super.new(pet_name, pet_age);  // Call parent constructor
    this.is_indoor = indoor;
  endfunction
  
  // Override make_sound method
  function string make_sound();
    return "Meow! Meow!";
  endfunction
  
  // Override get_description method
  function string get_description();
    string location = is_indoor ? "indoor" : "outdoor";
    return $sformatf("%s is a %d-year-old %s cat", name, age, location);
  endfunction
  
  // Cat-specific method
  function void purr();
    $display("%s is purring contentedly!", name);
  endfunction
endclass

// Top-level module for design
module pet_hierarchy_design;
  // This module serves as a container for the class definitions
  // The actual testing will be done in the testbench
endmodule
```

```systemverilog
// pet_hierarchy_design_testbench.sv
// Testbench for Basic Pet Hierarchy Example

module pet_hierarchy_testbench;
  
  // Instantiate the design module
  pet_hierarchy_design DESIGN_INSTANCE();
  
  // Test variables
  Animal base_animal;
  Dog my_dog;
  Cat my_cat;
  Animal animal_ref;  // Reference for polymorphism demonstration
  
  initial begin
    // Setup waveform dumping
    $dumpfile("pet_hierarchy_testbench.vcd");
    $dumpvars(0, pet_hierarchy_testbench);
    
    $display("=== Pet Hierarchy Inheritance Example ===");
    $display();
    
    // Test 1: Create base Animal
    $display("1. Testing Base Animal Class:");
    base_animal = new("Generic Pet", 5);
    base_animal.display_info();
    $display("Sound: %s", base_animal.make_sound());
    $display("Description: %s", base_animal.get_description());
    $display();
    
    // Test 2: Create Dog instance
    $display("2. Testing Dog Class (inherits from Animal):");
    my_dog = new("Buddy", 3, "Golden Retriever");
    my_dog.display_info();
    $display("Sound: %s", my_dog.make_sound());
    $display("Description: %s", my_dog.get_description());
    my_dog.wag_tail();
    $display();
    
    // Test 3: Create Cat instance
    $display("3. Testing Cat Class (inherits from Animal):");
    my_cat = new("Whiskers", 2, 1);  // Indoor cat
    my_cat.display_info();
    $display("Sound: %s", my_cat.make_sound());
    $display("Description: %s", my_cat.get_description());
    my_cat.purr();
    $display();
    
    // Test 4: Polymorphism demonstration
    $display("4. Testing Polymorphism:");
    $display("Using Animal reference to point to Dog:");
    animal_ref = my_dog;
    $display("Sound via Animal reference: %s", animal_ref.make_sound());
    $display("Description via Animal reference: %s", 
             animal_ref.get_description());
    
    $display("Using Animal reference to point to Cat:");
    animal_ref = my_cat;
    $display("Sound via Animal reference: %s", animal_ref.make_sound());
    $display("Description via Animal reference: %s", 
             animal_ref.get_description());
    $display();
    
    // Test 5: Create different pets with default constructors
    $display("5. Testing Default Constructors:");
    begin
      Dog default_dog;
      Cat default_cat;
      
      default_dog = new();
      default_cat = new();
      
      $display("Default Dog: %s", default_dog.get_description());
      $display("Default Cat: %s", default_cat.get_description());
    end
    
    $display();
    $display("=== All tests completed successfully! ===");
    
    // Small delay before finishing
    #10;
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Pet Hierarchy Inheritance Example ===

1. Testing Base Animal Class:
Pet Name: Generic Pet, Age:           5 years
Sound: Some generic animal sound
Description: Generic Pet is           5 years old

2. Testing Dog Class (inherits from Animal):
Pet Name: Buddy, Age:           3 years
Sound: Woof! Woof!
Description: Buddy is a           3-year-old Golden Retriever
Buddy is wagging tail happily!

3. Testing Cat Class (inherits from Animal):
Pet Name: Whiskers, Age:           2 years
Sound: Meow! Meow!
Description: Whiskers is a           2-year-old indoor cat
Whiskers is purring contentedly!

4. Testing Polymorphism:
Using Animal reference to point to Dog:
Sound via Animal reference: Woof! Woof!
Description via Animal reference: Buddy is a           3-year-old Golden
Retriever
Using Animal reference to point to Cat:
Sound via Animal reference: Meow! Meow!
Description via Animal reference: Whiskers is a           2-year-old indoor cat

5. Testing Default Co

0

### **Example 2:** Electronic Device Family
**Description:** Device base class with Phone and Laptop derived classes. Demonstrates inheriting common properties like brand, model, and power_on() method.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_2__electronic_device_family/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// electronic_device_family.sv
// Electronic Device Family with inheritance-like behavior

// Base device interface defining common properties and methods
interface device_interface;
  string brand;
  string model;
  bit power_state;
  
  // Common power on method
  task power_on();
    power_state = 1'b1;
    $display("Device %s %s powered ON", brand, model);
  endtask
  
  // Common power off method  
  task power_off();
    power_state = 1'b0;
    $display("Device %s %s powered OFF", brand, model);
  endtask
  
  // Pure virtual method to be implemented by derived classes
  task show_device_info();
    $display("Generic device: %s %s", brand, model);
  endtask
endinterface

// Phone class inheriting from device interface
module phone_device(device_interface dev_if);
  string phone_number = "+1-555-0123";
  bit has_camera = 1'b1;
  
  initial begin
    dev_if.brand = "Apple";
    dev_if.model = "iPhone 15";
    #1; // Small delay to ensure assignment
  end
  
  // Override device info method
  task show_phone_info();
    $display("Phone: %s %s", dev_if.brand, dev_if.model);
    $display("  Phone Number: %s", phone_number);
    $display("  Has Camera: %s", has_camera ? "Yes" : "No");
  endtask
  
  // Phone-specific method
  task make_call(string number);
    if (dev_if.power_state) begin
      $display("Calling %s from %s...", number, phone_number);
    end else begin
      $display("Cannot make call - phone is powered off");
    end
  endtask
endmodule

// Laptop class inheriting from device interface
module laptop_device(device_interface dev_if);
  string operating_system = "Ubuntu Linux";
  int ram_gb = 16;
  
  initial begin
    dev_if.brand = "Dell";
    dev_if.model = "XPS 13";
    #1; // Small delay to ensure assignment
  end
  
  // Override device info method
  task show_laptop_info();
    $display("Laptop: %s %s", dev_if.brand, dev_if.model);
    $display("  Operating System: %s", operating_system);
    $display("  RAM: %0d GB", ram_gb);
  endtask
  
  // Laptop-specific method
  task run_application(string app_name);
    if (dev_if.power_state) begin
      $display("Running %s on %s %s", app_name, dev_if.brand, 
               dev_if.model);
    end else begin
      $display("Cannot run application - laptop is powered off");
    end
  endtask
endmodule

// Top-level design module
module electronic_device_family();
  // Create device interfaces
  device_interface phone_if();
  device_interface laptop_if();
  
  // Instantiate device modules
  phone_device phone_inst(phone_if);
  laptop_device laptop_inst(laptop_if);
  
  initial begin
    $display("=== Electronic Device Family Demo ===");
    $display();
    
    // Wait for initialization
    #2;
    
    // Show initial device information
    phone_inst.show_phone_info();
    $display();
    laptop_inst.show_laptop_info();
    $display();
    
    // Test common power methods
    $display("--- Testing Power Control ---");
    phone_if.power_on();
    laptop_if.power_on();
    $display();
    
    // Test device-specific methods
    $display("--- Testing Device-Specific Methods ---");
    phone_inst.make_call("+1-555-9876");
    laptop_inst.run_application("Firefox Browser");
    $display();
    
    // Test power off
    $display("--- Testing Power Off ---");
    phone_if.power_off();
    laptop_if.power_off();
    $display();
    
    // Try to use devices when powered off
    $display("--- Testing Powered Off State ---");
    phone_inst.make_call("+1-555-5555");
    laptop_inst.run_application("Text Editor");
  end
endmodule
```

```systemverilog
// electronic_device_family_testbench.sv
// Testbench for Electronic Device Family demonstration

module device_family_testbench;
  // Instantiate the design under test
  electronic_device_family DEVICE_FAMILY_INSTANCE();
  
  initial begin
    // Dump waves for debugging
    $dumpfile("device_family_testbench.vcd");
    $dumpvars(0, device_family_testbench);
    
    // Wait for design to complete
    #100;
    
    $display();
    $display("=== Additional Testbench Verification ===");
    $display();
    
    // Test power state verification
    $display("--- Power State Verification ---");
    $display("Phone power state: %s", 
             DEVICE_FAMILY_INSTANCE.phone_if.power_state ? "ON" : "OFF");
    $display("Laptop power state: %s",
             DEVICE_FAMILY_INSTANCE.laptop_if.power_state ? "ON" : "OFF");
    $display();
    
    // Test brand and model access
    $display("--- Device Properties Access ---");
    $display("Phone brand: %s, model: %s", 
             DEVICE_FAMILY_INSTANCE.phone_if.brand,
             DEVICE_FAMILY_INSTANCE.phone_if.model);
    $display("Laptop brand: %s, model: %s",
             DEVICE_FAMILY_INSTANCE.laptop_if.brand,
             DEVICE_FAMILY_INSTANCE.laptop_if.model);
    $display();
    
    // Test polymorphic behavior
    $display("--- Polymorphic Method Testing ---");
    DEVICE_FAMILY_INSTANCE.phone_if.show_device_info();
    DEVICE_FAMILY_INSTANCE.laptop_if.show_device_info();
    $display();
    
    $display("Hello from device family testbench!");
    $display();
    
    // End simulation
    $finish;
  end
endmodule
```

Verilator Simulation Output:
=== Electronic Device Family Demo ===

Phone: Apple iPhone 15
  Phone Number: +1-555-0123
  Has Camera: Yes

Laptop: Dell XPS 13
  Operating System: Ubuntu Linux
  RAM: 16 GB

--- Testing Power Control ---
Device Apple iPhone 15 powered ON
Device Dell XPS 13 powered ON

--- Testing Device-Specific Methods ---
Calling +1-555-9876 from +1-555-0123...
Running Firefox Browser on Dell XPS 13

--- Testing Power Off ---
Device Apple iPhone 15 powered OFF
Device Dell XPS 13 powered OFF

--- Testing Powered Off State ---
Cannot make call - phone is powered off
Cannot run application - laptop is powered off

=== Additional Testbench Verification ===

--- Power State Verification ---
Phone power state: OFF
Laptop power state: OFF

--- Device Properties Access ---
Phone brand: Apple, model: iPhone 15
Laptop brand: Dell, model: XPS 13

--- Polymorphic Method Testing ---
Generic device: Apple iPhone 15
Generic device: Dell XPS 13

Hello from device family testbench!

Pro

0

### **Example 3:** Simple Food Chain
**Description:** Food base class with Fruit and Vegetable derived classes. Shows inheritance of basic properties like name, color, and nutritional_value.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_3__simple_food_chain/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// food_chain_classes.sv
package food_chain_pkg;

  // Base Food class
  virtual class Food;
    string name;
    string color;
    int nutritional_value;
    
    // Constructor
    function new(string n, string c, int nv);
      name = n;
      color = c;
      nutritional_value = nv;
    endfunction
    
    // Virtual method to display info
    virtual function void display_info();
      $display("Food: %s, Color: %s, Nutritional Value: %0d", 
               name, color, nutritional_value);
    endfunction
    
    // Virtual method to get food type
    virtual function string get_food_type();
      return "Generic Food";
    endfunction
  endclass

  // Fruit class derived from Food
  class Fruit extends Food;
    bit is_sweet;
    string season;
    
    // Constructor
    function new(string n, string c, int nv, bit sweet, string s);
      super.new(n, c, nv);
      is_sweet = sweet;
      season = s;
    endfunction
    
    // Override display_info method
    virtual function void display_info();
      $display("Fruit: %s, Color: %s, Nutritional Value: %0d, Sweet: %s, Season: %s",
               name, color, nutritional_value, 
               is_sweet ? "Yes" : "No", season);
    endfunction
    
    // Override get_food_type method
    virtual function string get_food_type();
      return "Fruit";
    endfunction
    
    // Fruit-specific method
    function void ripen();
      $display("%s is ripening...", name);
    endfunction
  endclass

  // Vegetable class derived from Food
  class Vegetable extends Food;
    string part_of_plant;
    bit needs_cooking;
    
    // Constructor
    function new(string n, string c, int nv, string part, bit cooking);
      super.new(n, c, nv);
      part_of_plant = part;
      needs_cooking = cooking;
    endfunction
    
    // Override display_info method
    virtual function void display_info();
      $display("Vegetable: %s, Color: %s, Nutritional Value: %0d, Part: %s, Needs Cooking: %s",
               name, color, nutritional_value, part_of_plant,
               needs_cooking ? "Yes" : "No");
    endfunction
    
    // Override get_food_type method
    virtual function string get_food_type();
      return "Vegetable";
    endfunction
    
    // Vegetable-specific method
    function void harvest();
      $display("%s is being harvested from the %s", name, part_of_plant);
    endfunction
  endclass

endpackage

module food_chain_classes();

  initial begin
    $display("Food Chain Classes - Design Module");
    $display("=====================================");
    $display("Classes defined in food_chain_pkg package");
    $display();
  end

endmodule
```

```systemverilog
// food_chain_classes_testbench.sv
module food_chain_testbench;

  // Import the food chain package
  import food_chain_pkg::*;

  // Instantiate design under test
  food_chain_classes FOOD_CHAIN_INSTANCE();

  // Declare all class handles and variables at module level
  Fruit apple, banana, orange;
  Vegetable carrot, spinach, potato;
  Food food_handle;
  food_chain_pkg::Fruit scoped_apple;
  int total_nutrition;

  // Task to create and initialize all food objects
  task create_food_objects();
    apple = new("Apple", "Red", 95, 1, "Fall");
    banana = new("Banana", "Yellow", 105, 1, "Year-round");
    orange = new("Orange", "Orange", 85, 1, "Winter");
    carrot = new("Carrot", "Orange", 25, "Root", 0);
    spinach = new("Spinach", "Green", 15, "Leaf", 0);
    potato = new("Potato", "Brown", 160, "Root", 1);
    scoped_apple = new("Scoped Apple", "Green", 80, 1, "Summer");
  endtask

  // Task to display fruit information
  task display_fruits();
    $display("Fruits Information:");
    $display("------------------");
    apple.display_info();
    banana.display_info();
    orange.display_info();
    $display();
  endtask

  // Task to display vegetable information
  task display_vegetables();
    $display("Vegetables Information:");
    $display("----------------------");
    carrot.display_info();
    spinach.display_info();
    potato.display_info();
    $display();
  endtask

  // Task to demonstrate fruit operations
  task demonstrate_fruit_operations();
    $display("Fruit Operations:");
    $display("----------------");
    apple.ripen();
    banana.ripen();
    orange.ripen();
    $display();
  endtask

  // Task to demonstrate vegetable operations
  task demonstrate_vegetable_operations();
    $display("Vegetable Operations:");
    $display("--------------------");
    carrot.harvest();
    spinach.harvest();
    potato.harvest();
    $display();
  endtask

  // Task to calculate and display nutritional information
  task display_nutrition_summary();
    $display("Nutritional Summary:");
    $display("-------------------");
    
    total_nutrition = 0;
    total_nutrition += apple.nutritional_value;
    $display("Apple: %0d calories", apple.nutritional_value);
    
    total_nutrition += banana.nutritional_value;
    $display("Banana: %0d calories", banana.nutritional_value);
    
    total_nutrition += orange.nutritional_value;
    $display("Orange: %0d calories", orange.nutritional_value);
    
    total_nutrition += carrot.nutritional_value;
    $display("Carrot: %0d calories", carrot.nutritional_value);
    
    total_nutrition += spinach.nutritional_value;
    $display("Spinach: %0d calories", spinach.nutritional_value);
    
    total_nutrition += potato.nutritional_value;
    $display("Potato: %0d calories", potato.nutritional_value);
    
    $display("Total Calories: %0d", total_nutrition);
    $display();
  endtask

  // Task to test polymorphism
  task test_polymorphism();
    $display("Polymorphism Test:");
    $display("-----------------");
    
    food_handle = apple;
    $display("Base handle pointing to apple:");
    $display("Type: %s", food_handle.get_food_type());
    food_handle.display_info();
    $display();
    
    food_handle = carrot;
    $display("Base handle pointing to carrot:");
    $display("Type: %s", food_handle.get_food_type());
    food_handle.display_info();
    $display();
  endtask

  // Task to test package scope
  task test_package_scope();
    $display("Package Scope Test:");
    $display("------------------");
    scoped_apple.display_info();
    $display();
  endtask

  // Main test sequence
  initial begin
    // Dump waves
    $dumpfile("food_chain_testbench.vcd");
    $dumpvars(0, food_chain_testbench);
    
    $display("Food Chain Inheritance Example");
    $display("==============================");
    $display();
    
    // Create all food objects
    create_food_objects();
    
    // Display information
    display_fruits();
    display_vegetables();
    
    // Demonstrate operations
    demonstrate_fruit_operations();
    demonstrate_vegetable_operations();
    
    // Show nutritional summary
    display_nutrition_summary();
    
    // Test polymorphism
    test_polymorphism();
    
    // Test package scope
    test_package_scope();
    
    $display("Food Chain Example Complete!");
    $display("============================");
    
    #1;  // Wait for a time unit
  end

endmodule
```

Verilator Simulation Output:
Food Chain Classes - Design Module
Classes defined in food_chain_pkg package

Food Chain Inheritance Example

Fruits Information:
------------------
Fruit: Apple, Color: Red, Nutritional Value: 95, Sweet: Yes, Season: Fall
Fruit: Banana, Color: Yellow, Nutritional Value: 105, Sweet: Yes, Season: Year-
round
Fruit: Orange, Color: Orange, Nutritional Value: 85, Sweet: Yes, Season: Winter

Vegetables Information:
----------------------
Vegetable: Carrot, Color: Orange, Nutritional Value: 25, Part: Root, Needs
Cooking:  No
Vegetable: Spinach, Color: Green, Nutritional Value: 15, Part: Leaf, Needs
Cooking:  No
Vegetable: Potato, Color: Brown, Nutritional Value: 160, Part: Root, Needs
Cooking: Yes

Fruit Operations:
----------------
Apple is ripening...
Banana is ripening...
Orange is ripening...

Vegetable Operations:
--------------------
Carrot is being harvested from the Root
Spinach is being harvested from the Leaf
Potato is being harvested from the Root

Nut

0

## Method Overriding

Method overriding allows derived classes to provide specific implementations of methods defined in their parent classes.

### Rules for Method Overriding

1. The method signature must match exactly
2. Use the `virtual` keyword in the base class
3. The derived class method automatically becomes virtual

### Example: Shape Hierarchy with Method Overriding

```systemverilog
// Base Shape class
class Shape;
    string name;
    
    function new(string n = "Shape");
        name = n;
    endfunction
    
    // Virtual method to be overridden
    virtual function real calculate_area();
        $display("Warning: calculate_area() not implemented for %s", name);
        return 0.0;
    endfunction
    
    virtual function void draw();
        $display("Drawing a generic %s", name);
    endfunction
endclass

// Rectangle class
class Rectangle extends Shape;
    real width, height;
    
    function new(real w = 1.0, real h = 1.0);
        super.new("Rectangle");
        width = w;
        height = h;
    endfunction
    
    // Override calculate_area method
    virtual function real calculate_area();
        return width * height;
    endfunction
    
    virtual function void draw();
        $display("Drawing rectangle: %0.2f x %0.2f", width, height);
    endfunction
endclass

// Circle class
class Circle extends Shape;
    real radius;
    
    function new(real r = 1.0);
        super.new("Circle");
        radius = r;
    endfunction
    
    virtual function real calculate_area();
        return 3.14159 * radius * radius;
    endfunction
    
    virtual function void draw();
        $display("Drawing circle with radius: %0.2f", radius);
    endfunction
endclass

// Triangle class
class Triangle extends Shape;
    real base, height;
    
    function new(real b = 1.0, real h = 1.0);
        super.new("Triangle");
        base = b;
        height = h;
    endfunction
    
    virtual function real calculate_area();
        return 0.5 * base * height;
    endfunction
    
    virtual function void draw();
        $display("Drawing triangle: base=%0.2f, height=%0.2f", base, height);
    endfunction
endclass
```

### **Example 4:** Sound Maker Animals
**Description:** Animal base class with make_sound() method. Dog, Cat, and Cow classes override to make "Woof!", "Meow!", and "Moo!" respectively.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_4__sound_maker_animals/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// animal_sound_maker.sv
package animal_sound_pkg;

  // Animal type constants
  parameter int ANIMAL_DOG = 0;
  parameter int ANIMAL_CAT = 1;
  parameter int ANIMAL_COW = 2;
  parameter int ANIMAL_GENERIC = 3;

  // Function to make sound based on animal type
  function automatic void make_sound(int animal_type, string name);
    case (animal_type)
      ANIMAL_DOG: $display("%s says: Woof!", name);
      ANIMAL_CAT: $display("%s says: Meow!", name);
      ANIMAL_COW: $display("%s says: Moo!", name);
      ANIMAL_GENERIC: $display("%s makes a generic animal sound", name);
      default: $display("%s makes an unknown sound", name);
    endcase
  endfunction

  // Function to introduce animal
  function automatic void introduce(int animal_type, string name);
    string type_str;
    case (animal_type)
      ANIMAL_DOG: type_str = "Dog";
      ANIMAL_CAT: type_str = "Cat";
      ANIMAL_COW: type_str = "Cow";
      ANIMAL_GENERIC: type_str = "Animal";
      default: type_str = "Unknown";
    endcase
    $display("I am %s, a %s", name, type_str);
  endfunction

  // Function to get animal type string
  function automatic string get_type_string(int animal_type);
    case (animal_type)
      ANIMAL_DOG: return "Dog";
      ANIMAL_CAT: return "Cat";
      ANIMAL_COW: return "Cow";
      ANIMAL_GENERIC: return "Animal";
      default: return "Unknown";
    endcase
  endfunction

endpackage

module animal_sound_maker();
  import animal_sound_pkg::*;

  initial begin
    $display("=== Animal Sound Maker Demo ===");
    $display();
  end

endmodule
```

```systemverilog
// animal_sound_maker_testbench.sv
module animal_sound_testbench;
  import animal_sound_pkg::*;

  // Instantiate design under test
  animal_sound_maker ANIMAL_SOUND_INSTANCE();

  // Declare all variables at module level for Verilator compatibility
  int buddy_type;
  string buddy_name;
  int whiskers_type;
  string whiskers_name;
  int bessie_type;
  string bessie_name;
  
  // Arrays for farm animals
  int farm_animal_types[3];
  string farm_animal_names[3];
  int i;

  initial begin
    // Dump waves for debugging
    $dumpfile("animal_sound_testbench.vcd");
    $dumpvars(0, animal_sound_testbench);
    
    $display("=== Creating Animal Objects ===");
    buddy_type = ANIMAL_DOG;
    buddy_name = "Buddy";
    
    whiskers_type = ANIMAL_CAT;
    whiskers_name = "Whiskers";
    
    bessie_type = ANIMAL_COW;
    bessie_name = "Bessie";
    
    $display();
    $display("=== Individual Animal Sounds ===");
    
    // Test individual animals
    introduce(buddy_type, buddy_name);
    make_sound(buddy_type, buddy_name);
    $display();
    
    introduce(whiskers_type, whiskers_name);
    make_sound(whiskers_type, whiskers_name);
    $display();
    
    introduce(bessie_type, bessie_name);
    make_sound(bessie_type, bessie_name);
    $display();
    
    // Demonstrate polymorphism with arrays
    $display("=== Polymorphism Demo ===");
    farm_animal_types[0] = buddy_type;
    farm_animal_names[0] = buddy_name;
    farm_animal_types[1] = whiskers_type;
    farm_animal_names[1] = whiskers_name;
    farm_animal_types[2] = bessie_type;
    farm_animal_names[2] = bessie_name;
    
    $display("Farm animals making sounds:");
    for (i = 0; i < 3; i = i + 1) begin
      make_sound(farm_animal_types[i], farm_animal_names[i]);
    end
    
    $display();
    $display("=== Sound Maker Animals Test Complete ===");
    
    #10; // Wait a bit before finishing
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Animal Sound Maker Demo ===

=== Creating Animal Objects ===

=== Individual Animal Sounds ===
I am Buddy, a Dog
Buddy says: Woof!

I am Whiskers, a Cat
Whiskers says: Meow!

I am Bessie, a Cow
Bessie says: Moo!

=== Polymorphism Demo ===
Farm animals making sounds:
Buddy says: Woof!
Whiskers says: Meow!
Bessie says: Moo!

=== Sound Maker Animals Test Complete ===
Process finished with return code: 0
Removing Chapter_10_examples/example_4__sound_maker_animals/obj_dir directory...
Chapter_10_examples/example_4__sound_maker_animals/obj_dir removed successfully.


0

### **Example 5:** Transportation Methods
**Description:** Transport base class with move() method. Car, Bike, and Plane classes override to show "Driving...", "Pedaling...", and "Flying...".

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_5__transportation_methods/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// transportation_methods.sv
// Transportation base class and derived classes example

// Base Transportation class
virtual class Transportation;
  string name;
  
  function new(string transport_name);
    this.name = transport_name;
  endfunction
  
  // Virtual method to be overridden by derived classes
  virtual function void move();
    $display("Generic transportation moving...");
  endfunction
  
  virtual function void show_info();
    $display("Transportation: %s", name);
  endfunction
endclass

// Car class - derived from Transportation
class Car extends Transportation;
  function new(string car_name);
    super.new(car_name);
  endfunction
  
  // Override move method
  virtual function void move();
    $display("Driving...");
  endfunction
endclass

// Bike class - derived from Transportation
class Bike extends Transportation;
  function new(string bike_name);
    super.new(bike_name);
  endfunction
  
  // Override move method
  virtual function void move();
    $display("Pedaling...");
  endfunction
endclass

// Plane class - derived from Transportation
class Plane extends Transportation;
  function new(string plane_name);
    super.new(plane_name);
  endfunction
  
  // Override move method
  virtual function void move();
    $display("Flying...");
  endfunction
endclass

// Transportation manager module
module transportation_manager();
  Car my_car;
  Bike my_bike;
  Plane my_plane;
  
  initial begin
    $display("=== Transportation Methods Example ===");
    $display();
    
    // Create instances of different transportation types
    my_car = new("Toyota Camry");
    my_bike = new("Mountain Bike");
    my_plane = new("Boeing 737");
    
    // Show info and demonstrate move methods
    my_car.show_info();
    my_car.move();
    $display();
    
    my_bike.show_info();
    my_bike.move();
    $display();
    
    my_plane.show_info();
    my_plane.move();
    $display();
    
    $display("=== Polymorphism Example ===");
    demonstrate_polymorphism();
  end
  
  // Function to demonstrate polymorphism
  function void demonstrate_polymorphism();
    Transportation transports[];
    Car poly_car;
    Bike poly_bike;
    Plane poly_plane;
    
    // Create array of different transportation types
    transports = new[3];
    
    // Create individual instances first
    poly_car = new("Honda Civic");
    poly_bike = new("Road Bike");
    poly_plane = new("Airbus A320");
    
    // Assign to base class array
    transports[0] = poly_car;
    transports[1] = poly_bike;
    transports[2] = poly_plane;
    
    // Call move() on each - polymorphic behavior
    foreach(transports[i]) begin
      transports[i].show_info();
      transports[i].move();
      $display();
    end
  endfunction
endmodule
```

```systemverilog
// transportation_methods_testbench.sv
// Testbench for transportation methods example

module transportation_testbench;
  // Instantiate the transportation manager
  transportation_manager TRANSPORT_INSTANCE();
  
  initial begin
    // Dump waves for Verilator
    $dumpfile("transportation_testbench.vcd");
    $dumpvars(0, transportation_testbench);
    
    #1; // Wait for design to execute
    
    $display("=== Additional Transportation Tests ===");
    test_individual_transports();
    
    $display("=== Test Complete ===");
    $display();
  end
  
  // Additional test function
  function void test_individual_transports();
    Car test_car;
    Bike test_bike;
    Plane test_plane;
    
    $display("Testing individual transportation objects:");
    $display();
    
    // Test Car
    test_car = new("Test Car");
    $display("Creating and testing %s:", test_car.name);
    test_car.move();
    $display();
    
    // Test Bike
    test_bike = new("Test Bike");
    $display("Creating and testing %s:", test_bike.name);
    test_bike.move();
    $display();
    
    // Test Plane
    test_plane = new("Test Plane");
    $display("Creating and testing %s:", test_plane.name);
    test_plane.move();
    $display();
  endfunction
endmodule
```

Verilator Simulation Output:
=== Transportation Methods Example ===

Transportation: Toyota Camry
Driving...

Transportation: Mountain Bike
Pedaling...

Transportation: Boeing 737
Flying...

=== Polymorphism Example ===
Transportation: Honda Civic
Driving...

Transportation: Road Bike
Pedaling...

Transportation: Airbus A320
Flying...

=== Additional Transportation Tests ===
Testing individual transportation objects:

Creating and testing Test Car:
Driving...

Creating and testing Test Bike:
Pedaling...

Creating and testing Test Plane:
Flying...

=== Test Complete ===
Process finished with return code: 0
Removing Chapter_10_examples/example_5__transportation_methods/obj_dir directory...
Chapter_10_examples/example_5__transportation_methods/obj_dir removed successfully.


0

### **Example 6:** Simple Math Operations
**Description:** Calculator base class with calculate() method. Add, Subtract, and Multiply classes override to perform specific operations.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_6__simple_math_operations/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// simple_math_calculator.sv
module simple_math_calculator_module();

  // Base Calculator class with virtual calculate method
  virtual class Calculator;
    protected int operand_a;
    protected int operand_b;
    
    function new(int a, int b);
      operand_a = a;
      operand_b = b;
    endfunction
    
    // Virtual method to be overridden by derived classes
    pure virtual function int calculate();
    
    function void display_operands();
      $display("Operand A: %0d, Operand B: %0d", operand_a, operand_b);
    endfunction
  endclass

  // Add class - performs addition
  class Add extends Calculator;
    function new(int a, int b);
      super.new(a, b);
    endfunction
    
    virtual function int calculate();
      return operand_a + operand_b;
    endfunction
  endclass

  // Subtract class - performs subtraction
  class Subtract extends Calculator;
    function new(int a, int b);
      super.new(a, b);
    endfunction
    
    virtual function int calculate();
      return operand_a - operand_b;
    endfunction
  endclass

  // Multiply class - performs multiplication
  class Multiply extends Calculator;
    function new(int a, int b);
      super.new(a, b);
    endfunction
    
    virtual function int calculate();
      return operand_a * operand_b;
    endfunction
  endclass

  initial begin
    Add add_calc;
    Subtract sub_calc;
    Multiply mul_calc;
    Calculator calc_handle;
    int result;
    
    $display();
    $display("=== Simple Math Operations Example ===");
    $display();
    
    // Test Addition
    add_calc = new(15, 7);
    calc_handle = add_calc;
    calc_handle.display_operands();
    result = calc_handle.calculate();
    $display("Addition Result: %0d", result);
    $display();
    
    // Test Subtraction
    sub_calc = new(20, 8);
    calc_handle = sub_calc;
    calc_handle.display_operands();
    result = calc_handle.calculate();
    $display("Subtraction Result: %0d", result);
    $display();
    
    // Test Multiplication
    mul_calc = new(6, 9);
    calc_handle = mul_calc;
    calc_handle.display_operands();
    result = calc_handle.calculate();
    $display("Multiplication Result: %0d", result);
    $display();
    
    $display("=== Math Operations Complete ===");
  end

endmodule
```

```systemverilog
// simple_math_calculator_testbench.sv
module test_bench_module;

  // Instantiate design under test
  simple_math_calculator_module CALCULATOR_INSTANCE();

  initial begin
    // Dump waves for debugging
    $dumpfile("test_bench_module.vcd");
    $dumpvars(0, test_bench_module);
    
    $display();
    $display("=== Math Calculator Testbench Started ===");
    $display();
    
    // Wait for design to complete
    #10;
    
    $display();
    $display("=== All Math Operations Verified ===");
    $display();
    
    // End simulation
    $finish;
  end

endmodule
```

Verilator Simulation Output:

=== Math Calculator Testbench Started ===


=== Simple Math Operations Example ===

Operand A: 15, Operand B: 7
Addition Result: 22

Operand A: 20, Operand B: 8
Subtraction Result: 12

Operand A: 6, Operand B: 9
Multiplication Result: 54

=== Math Operations Complete ===

=== All Math Operations Verified ===

Process finished with return code: 0
Removing Chapter_10_examples/example_6__simple_math_operations/obj_dir directory...
Chapter_10_examples/example_6__simple_math_operations/obj_dir removed successfully.


0

## The super Keyword

The `super` keyword provides access to the parent class's methods and properties from within a derived class.

### Uses of super

1. **Calling parent constructor**: `super.new()`
2. **Calling parent methods**: `super.method_name()`
3. **Accessing parent properties**: `super.property_name`

### Example: Employee Hierarchy

```systemverilog
class Employee;
    string name;
    int employee_id;
    real base_salary;
    
    function new(string n, int id, real salary);
        name = n;
        employee_id = id;
        base_salary = salary;
    endfunction
    
    virtual function real calculate_pay();
        return base_salary;
    endfunction
    
    virtual function void display_info();
        $display("Employee: %s (ID: %0d), Base Salary: $%0.2f", 
                 name, employee_id, base_salary);
    endfunction
endclass

class Manager extends Employee;
    real bonus_percentage;
    int team_size;
    
    function new(string n, int id, real salary, real bonus = 0.15, int team = 5);
        super.new(n, id, salary);  // Call parent constructor
        bonus_percentage = bonus;
        team_size = team;
    endfunction
    
    virtual function real calculate_pay();
        real base_pay = super.calculate_pay();  // Get base salary from parent
        return base_pay + (base_pay * bonus_percentage);
    endfunction
    
    virtual function void display_info();
        super.display_info();  // Call parent display method
        $display("  Role: Manager, Team Size: %0d, Bonus: %0.1f%%", 
                 team_size, bonus_percentage * 100);
    endfunction
    
    function void conduct_meeting();
        $display("%s is conducting a team meeting", name);
    endfunction
endclass

class Developer extends Employee;
    string programming_language;
    int projects_completed;
    
    function new(string n, int id, real salary, string lang = "SystemVerilog");
        super.new(n, id, salary);
        programming_language = lang;
        projects_completed = 0;
    endfunction
    
    virtual function real calculate_pay();
        real base_pay = super.calculate_pay();
        real project_bonus = projects_completed * 500.0;  // $500 per project
        return base_pay + project_bonus;
    endfunction
    
    virtual function void display_info();
        super.display_info();
        $display("  Role: Developer, Language: %s, Projects: %0d", 
                 programming_language, projects_completed);
    endfunction
    
    function void complete_project();
        projects_completed++;
        $display("%s completed a project in %s", name, programming_language);
    endfunction
endclass
```

### **Example 7:** Student Grade System
**Description:** Student base class with name and basic info. GraduateStudent extends it, using super.new() to call parent constructor and super methods for grade calculation.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_7__student_grade_system/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// student_grade_system.sv

// Base Student class package
package student_pkg;

class Student;
  protected string name;
  protected int age;
  protected real base_grade;
  
  // Constructor
  function new(string student_name, int student_age, real grade);
    this.name = student_name;
    this.age = student_age;
    this.base_grade = grade;
    $display("Student created: %s, Age: %0d", name, age);
  endfunction
  
  // Virtual method for grade calculation
  virtual function real calculate_final_grade();
    return base_grade;
  endfunction
  
  // Method to display student info
  virtual function void display_info();
    $display("Student: %s, Age: %0d, Final Grade: %.2f", 
             name, age, calculate_final_grade());
  endfunction
  
  // Getter methods
  function string get_name();
    return name;
  endfunction
  
  function int get_age();
    return age;
  endfunction
  
endclass

endpackage : student_pkg

// Graduate Student class package
package graduate_student_pkg;

// Import the base student package
import student_pkg::*;

class GraduateStudent extends Student;
  protected real research_bonus;
  protected real thesis_score;
  
  // Constructor using super.new()
  function new(string student_name, int student_age, real grade, 
               real bonus, real thesis);
    super.new(student_name, student_age, grade);
    this.research_bonus = bonus;
    this.thesis_score = thesis;
    $display("Graduate Student specialized initialization complete");
  endfunction
  
  // Override grade calculation using super method
  virtual function real calculate_final_grade();
    real base_result;
    real final_result;
    
    // Call parent method using super
    base_result = super.calculate_final_grade();
    
    // Add graduate-specific calculations
    final_result = base_result + research_bonus + (thesis_score * 0.1);
    
    // Cap at 100
    if (final_result > 100.0) final_result = 100.0;
    
    return final_result;
  endfunction
  
  // Override display method using super
  virtual function void display_info();
    $display("=== Graduate Student Info ===");
    super.display_info();
    $display("Research Bonus: %.2f, Thesis Score: %.2f", 
             research_bonus, thesis_score);
  endfunction
  
endclass

endpackage : graduate_student_pkg
```

```systemverilog
// student_grade_system_testbench.sv
module student_grade_testbench;
  
  // Import both packages
  import student_pkg::*;
  import graduate_student_pkg::*;
  
  // Test variables
  Student regular_student;
  GraduateStudent grad_student;
  
  initial begin
    // Dump waves
    $dumpfile("student_grade_testbench.vcd");
    $dumpvars(0, student_grade_testbench);
    
    $display("=== Student Grade System Test ===");
    $display();
    
    // Test 1: Create regular student
    $display("Test 1: Creating regular student");
    regular_student = new("Alice Johnson", 20, 85.5);
    regular_student.display_info();
    $display();
    
    // Test 2: Create graduate student using super.new()
    $display("Test 2: Creating graduate student");
    grad_student = new("Bob Smith", 24, 82.0, 5.0, 90.0);
    grad_student.display_info();
    $display();
    
    // Test 3: Compare grade calculations
    $display("Test 3: Grade comparison");
    $display("Regular student '%s' final grade: %.2f", 
             regular_student.get_name(), 
             regular_student.calculate_final_grade());
    $display("Graduate student '%s' final grade: %.2f", 
             grad_student.get_name(), 
             grad_student.calculate_final_grade());
    $display();
    
    // Test 4: Polymorphism test
    $display("Test 4: Polymorphism test");
    begin
      Student student_ref;
      
      // Point to regular student
      student_ref = regular_student;
      $display("Via Student reference (regular):");
      student_ref.display_info();
      $display();
      
      // Point to graduate student
      student_ref = grad_student;
      $display("Via Student reference (graduate):");
      student_ref.display_info();
      $display();
    end
    
    // Test 5: Edge case - high grades
    $display("Test 5: High grade capping test");
    begin
      GraduateStudent high_achiever;
      high_achiever = new("Charlie Brown", 26, 95.0, 8.0, 95.0);
      high_achiever.display_info();
      $display();
    end
    
    $display("=== All tests completed ===");
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:

Test 1: Creating regular student
Student created: Alice Johnson, Age: 20
Student: Alice Johnson, Age: 20, Final Grade: 85.50

Test 2: Creating graduate student
Student created: Bob Smith, Age: 24
Graduate Student specialized initialization complete
=== Graduate Student Info ===
Student: Bob Smith, Age: 24, Final Grade: 96.00
Research Bonus: 5.00, Thesis Score: 90.00

Test 3: Grade comparison
Regular student 'Alice Johnson' final grade: 85.50
Graduate student 'Bob Smith' final grade: 96.00

Test 4: Polymorphism test
Via Student reference (regular):
Student: Alice Johnson, Age: 20, Final Grade: 85.50

Via Student reference (graduate):
=== Graduate Student Info ===
Student: Bob Smith, Age: 24, Final Grade: 96.00
Research Bonus: 5.00, Thesis Score: 90.00

Test 5: High grade capping test
Student created: Charlie Brown, Age: 26
Graduate Student specialized initialization complete
=== Graduate Student Info ===
Student: Charlie Brown, Age: 26, Final Grade: 100.00


0

### **Example 8:** Bank Account Hierarchy
**Description:** Account base class with balance operations. SavingsAccount uses super to call parent methods while adding interest calculation.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_8__bank_account_hierarchy/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// bank_account_hierarchy.sv
package bank_account_pkg;

  // Base Account class
  class Account;
    protected real balance;
    protected string account_number;
    
    // Constructor
    function new(string acc_num = "00000", real initial_balance = 0.0);
      account_number = acc_num;
      balance = initial_balance;
      $display("Account created: %s with balance $%.2f", 
               account_number, balance);
    endfunction
    
    // Deposit money
    virtual function void deposit(real amount);
      if (amount > 0) begin
        balance += amount;
        $display("Deposited $%.2f to account %s. New balance: $%.2f",
                 amount, account_number, balance);
      end else begin
        $display("Error: Deposit amount must be positive");
      end
    endfunction
    
    // Withdraw money
    virtual function bit withdraw(real amount);
      if (amount > 0 && amount <= balance) begin
        balance -= amount;
        $display("Withdrew $%.2f from account %s. New balance: $%.2f",
                 amount, account_number, balance);
        return 1;
      end else begin
        $display("Error: Insufficient funds or invalid amount");
        return 0;
      end
    endfunction
    
    // Get current balance
    virtual function real get_balance();
      return balance;
    endfunction
    
    // Display account info
    virtual function void display_info();
      $display("Account: %s | Balance: $%.2f", account_number, balance);
    endfunction
  endclass

  // SavingsAccount class inherits from Account
  class SavingsAccount extends Account;
    protected real interest_rate;
    
    // Constructor
    function new(string acc_num = "SAV00000", real initial_balance = 0.0,
                 real rate = 0.02);
      super.new(acc_num, initial_balance);  // Call parent constructor
      interest_rate = rate;
      $display("Savings account created with %.2f%% interest rate",
               interest_rate * 100);
    endfunction
    
    // Override deposit to show savings account specific behavior
    virtual function void deposit(real amount);
      super.deposit(amount);  // Call parent deposit method
      $display("Thank you for saving with us!");
    endfunction
    
    // Add interest calculation method
    function void calculate_interest();
      real interest = balance * interest_rate;
      super.deposit(interest);  // Use parent deposit to add interest
      $display("Interest calculated: $%.2f added to savings account",
               interest);
    endfunction
    
    // Override display_info to show interest rate
    virtual function void display_info();
      super.display_info();  // Call parent display method
      $display("Interest Rate: %.2f%%", interest_rate * 100);
    endfunction
    
    // Set new interest rate
    function void set_interest_rate(real new_rate);
      if (new_rate >= 0) begin
        interest_rate = new_rate;
        $display("Interest rate updated to %.2f%%", interest_rate * 100);
      end else begin
        $display("Error: Interest rate cannot be negative");
      end
    endfunction
  endclass

endpackage

module bank_account_hierarchy_module;
  import bank_account_pkg::*;
  
  // Module just imports the package - testing done in testbench
  initial begin
    $display("Bank Account Hierarchy module loaded");
  end
  
endmodule
```

```systemverilog
// bank_account_hierarchy_testbench.sv
module bank_account_test_module;
  import bank_account_pkg::*;
  
  // Instantiate design under test
  bank_account_hierarchy_module BANK_ACCOUNT_INSTANCE();
  
  // Test variables at module level
  Account basic_account;
  SavingsAccount savings_account;
  Account poly_account;
  
  initial begin
    // Dump waves
    $dumpfile("bank_account_test_module.vcd");
    $dumpvars(0, bank_account_test_module);
    
    $display("=== Bank Account Hierarchy Test ===");
    $display();
    
    // Test 1: Create basic account
    $display("--- Test 1: Basic Account Operations ---");
    basic_account = new("ACC12345", 1000.0);
    basic_account.display_info();
    basic_account.deposit(250.0);
    /* verilator lint_off IGNOREDRETURN */
    basic_account.withdraw(150.0);
    /* verilator lint_on IGNOREDRETURN */
    basic_account.display_info();
    $display();
    
    // Test 2: Create savings account
    $display("--- Test 2: Savings Account Operations ---");
    savings_account = new("SAV67890", 500.0, 0.05);
    savings_account.display_info();
    savings_account.deposit(200.0);
    savings_account.calculate_interest();
    savings_account.display_info();
    $display();
    
    // Test 3: Demonstrate inheritance and super usage
    $display("--- Test 3: Inheritance and Super Keyword ---");
    $display("Depositing to savings account (shows super.deposit()):");
    savings_account.deposit(100.0);
    $display();
    
    // Test 4: Interest rate operations
    $display("--- Test 4: Interest Rate Management ---");
    savings_account.set_interest_rate(0.03);
    savings_account.calculate_interest();
    savings_account.display_info();
    $display();
    
    // Test 5: Error handling
    $display("--- Test 5: Error Handling ---");
    basic_account.deposit(-50.0);  // Should show error
    /* verilator lint_off IGNOREDRETURN */
    basic_account.withdraw(2000.0);  // Should show error
    /* verilator lint_on IGNOREDRETURN */
    savings_account.set_interest_rate(-0.01);  // Should show error
    $display();
    
    // Test 6: Polymorphism demonstration
    $display("--- Test 6: Polymorphism ---");
    poly_account = savings_account;  // Assign savings account to base ref
    poly_account.display_info();  // Should call overridden method
    $display();
    
    $display("=== All tests completed ===");
    $display();
    
    #10;  // Wait for a few time units
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
Bank Account Hierarchy module loaded
=== Bank Account Hierarchy Test ===

--- Test 1: Basic Account Operations ---
Account created: ACC12345 with balance $1000.00
Account: ACC12345 | Balance: $1000.00
Deposited $250.00 to account ACC12345. New balance: $1250.00
Withdrew $150.00 from account ACC12345. New balance: $1100.00
Account: ACC12345 | Balance: $1100.00

--- Test 2: Savings Account Operations ---
Account created: SAV67890 with balance $500.00
Savings account created with 5.00% interest rate
Account: SAV67890 | Balance: $500.00
Interest Rate: 5.00%
Deposited $200.00 to account SAV67890. New balance: $700.00
Thank you for saving with us!
Deposited $35.00 to account SAV67890. New balance: $735.00
Interest calculated: $35.00 added to savings account
Account: SAV67890 | Balance: $735.00
Interest Rate: 5.00%

--- Test 3: Inheritance and Super Keyword ---
Depositing to savings account (shows super.deposit()):
Deposited $100.00 to account SAV67890. New balanc

0

### **Example 9:** Simple Game Characters
**Description:** Character base class with health and name. Warrior class uses super to initialize base stats then adds specific warrior abilities.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_9__simple_game_characters/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// game_characters.sv
package game_characters_pkg;

  // Base character class with health and name
  class Character;
    protected string name;
    protected int health;
    protected int max_health;
    
    // Constructor with name and initial health
    function new(string char_name, int initial_health);
      this.name = char_name;
      this.health = initial_health;
      this.max_health = initial_health;
    endfunction
    
    // Method to get character name
    virtual function string get_name();
      return this.name;
    endfunction
    
    // Method to get current health
    virtual function int get_health();
      return this.health;
    endfunction
    
    // Method to get max health
    virtual function int get_max_health();
      return this.max_health;
    endfunction
    
    // Method to take damage
    virtual function void take_damage(int damage);
      this.health -= damage;
      if (this.health < 0) this.health = 0;
      $display("%s takes %0d damage! Health: %0d/%0d", 
               this.name, damage, this.health, this.max_health);
    endfunction
    
    // Method to heal
    virtual function void heal(int amount);
      this.health += amount;
      if (this.health > this.max_health) this.health = this.max_health;
      $display("%s heals %0d points! Health: %0d/%0d", 
               this.name, amount, this.health, this.max_health);
    endfunction
    
    // Method to check if character is alive
    virtual function bit is_alive();
      return (this.health > 0);
    endfunction
    
    // Method to display character info
    virtual function void display_info();
      $display("Character: %s, Health: %0d/%0d", 
               this.name, this.health, this.max_health);
    endfunction
    
  endclass

  // Warrior class inheriting from Character
  class Warrior extends Character;
    protected int attack_power;
    protected int armor;
    
    // Constructor using super to initialize base class
    function new(string warrior_name, int initial_health, 
                 int warrior_attack, int warrior_armor);
      super.new(warrior_name, initial_health);  // Call parent constructor
      this.attack_power = warrior_attack;
      this.armor = warrior_armor;
    endfunction
    
    // Method to get attack power
    virtual function int get_attack_power();
      return this.attack_power;
    endfunction
    
    // Method to get armor value
    virtual function int get_armor();
      return this.armor;
    endfunction
    
    // Override take_damage to account for armor
    virtual function void take_damage(int damage);
      int actual_damage = damage - this.armor;
      if (actual_damage < 0) actual_damage = 0;
      
      this.health -= actual_damage;
      if (this.health < 0) this.health = 0;
      
      $display("%s blocks %0d damage with armor! Takes %0d damage!", 
               this.name, this.armor, actual_damage);
      $display("%s Health: %0d/%0d", this.name, this.health, 
               this.max_health);
    endfunction
    
    // Warrior-specific attack method
    virtual function void attack(Character target);
      if (target == null) begin
        $display("%s swings at nothing!", this.name);
        return;
      end
      
      $display("%s attacks %s with %0d attack power!", 
               this.name, target.get_name(), this.attack_power);
      target.take_damage(this.attack_power);
    endfunction
    
    // Override display_info to show warrior stats
    virtual function void display_info();
      $display("Warrior: %s, Health: %0d/%0d, Attack: %0d, Armor: %0d", 
               this.name, this.health, this.max_health, 
               this.attack_power, this.armor);
    endfunction
    
  endclass

endpackage

module game_characters_module;
  import game_characters_pkg::*;
  
  // Module instantiation for synthesis
  initial begin
    $display("Game Characters Module - Classes defined in package");
  end
  
endmodule
```

```systemverilog
// game_characters_testbench.sv
module game_characters_test_bench;
  import game_characters_pkg::*;
  
  // Instantiate the design module
  game_characters_module GAME_CHARACTERS_INSTANCE();
  
  // Test variables
  Character basic_char;
  Warrior warrior_char;
  Warrior enemy_warrior;
  Character char_ref;
  
  initial begin
    // Dump waves
    $dumpfile("game_characters_test_bench.vcd");
    $dumpvars(0, game_characters_test_bench);
    
    $display("=== Simple Game Characters Test ===");
    $display();
    
    // Test 1: Create basic character
    $display("--- Test 1: Basic Character ---");
    basic_char = new("Adventurer", 100);
    basic_char.display_info();
    $display("Is alive: %b", basic_char.is_alive());
    $display();
    
    // Test 2: Character takes damage and heals
    $display("--- Test 2: Character Damage and Healing ---");
    basic_char.take_damage(30);
    basic_char.heal(15);
    basic_char.display_info();
    $display();
    
    // Test 3: Create warrior using super constructor
    $display("--- Test 3: Warrior Creation ---");
    warrior_char = new("Sir Lancelot", 150, 25, 5);
    warrior_char.display_info();
    $display();
    
    // Test 4: Warrior takes damage (armor reduces damage)
    $display("--- Test 4: Warrior Armor Protection ---");
    warrior_char.take_damage(20);  // Should take 15 damage (20-5 armor)
    warrior_char.display_info();
    $display();
    
    // Test 5: Create enemy warrior for combat
    $display("--- Test 5: Warrior vs Warrior Combat ---");
    enemy_warrior = new("Dark Knight", 120, 20, 3);
    enemy_warrior.display_info();
    $display();
    
    // Test 6: Combat simulation
    $display("--- Test 6: Combat Simulation ---");
    $display("Round 1:");
    warrior_char.attack(enemy_warrior);
    $display();
    
    $display("Round 2:");
    enemy_warrior.attack(warrior_char);
    $display();
    
    $display("Round 3:");
    warrior_char.attack(enemy_warrior);
    $display();
    
    // Test 7: Character death
    $display("--- Test 7: Character Death ---");
    enemy_warrior.take_damage(100);  // Finish off enemy
    $display("Enemy is alive: %b", enemy_warrior.is_alive());
    $display();
    
    // Test 8: Attack on dead character
    $display("--- Test 8: Attack on Defeated Enemy ---");
    warrior_char.attack(enemy_warrior);
    $display();
    
    // Test 9: Healing limits
    $display("--- Test 9: Healing Limits ---");
    warrior_char.heal(200);  // Should cap at max health
    warrior_char.display_info();
    $display();
    
    // Test 10: Polymorphism test
    $display("--- Test 10: Polymorphism Test ---");
    char_ref = warrior_char;  // Base class reference to derived object
    char_ref.display_info();  // Should call warrior's display_info
    $display();
    
    $display("=== All Tests Complete ===");
    
    #1;  // Wait for a time unit
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
Game Characters Module - Classes defined in package
=== Simple Game Characters Test ===

--- Test 1: Basic Character ---
Character: Adventurer, Health: 100/100
Is alive: 1

--- Test 2: Character Damage and Healing ---
Adventurer takes 30 damage! Health: 70/100
Adventurer heals 15 points! Health: 85/100
Character: Adventurer, Health: 85/100

--- Test 3: Warrior Creation ---
Warrior: Sir Lancelot, Health: 150/150, Attack: 25, Armor: 5

--- Test 4: Warrior Armor Protection ---
Sir Lancelot blocks 5 damage with armor! Takes 15 damage!
Sir Lancelot Health: 135/150
Warrior: Sir Lancelot, Health: 135/150, Attack: 25, Armor: 5

--- Test 5: Warrior vs Warrior Combat ---
Warrior: Dark Knight, Health: 120/120, Attack: 20, Armor: 3

--- Test 6: Combat Simulation ---
Round 1:
Sir Lancelot attacks Dark Knight with 25 attack power!
Dark Knight blocks 3 damage with armor! Takes 22 damage!
Dark Knight Health: 98/120

Round 2:
Dark Knight attacks Sir Lancelot with 20 attack 

0

## Virtual Methods

Virtual methods enable polymorphism by allowing method calls to be resolved at runtime based on the actual object type.

### Virtual Method Rules

1. Use `virtual` keyword in the base class method declaration
2. Derived class methods that override virtual methods are automatically virtual
3. Virtual methods enable dynamic binding

### Example: Communication Protocol Stack

```systemverilog
// Base Protocol class
class Protocol;
    string protocol_name;
    int header_size;
    
    function new(string name = "Generic", int hdr_size = 0);
        protocol_name = name;
        header_size = hdr_size;
    endfunction
    
    // Virtual methods for protocol operations
    virtual function void encode_packet(ref bit [7:0] data[]);
        $display("Generic encoding for %s protocol", protocol_name);
    endfunction
    
    virtual function void decode_packet(ref bit [7:0] data[]);
        $display("Generic decoding for %s protocol", protocol_name);
    endfunction
    
    virtual function int get_overhead();
        return header_size;
    endfunction
    
    virtual function void display_info();
        $display("Protocol: %s, Header Size: %0d bytes", protocol_name, header_size);
    endfunction
endclass

// TCP Protocol
class TCP_Protocol extends Protocol;
    int sequence_number;
    int window_size;
    
    function new();
        super.new("TCP", 20);  // TCP header is 20 bytes minimum
        sequence_number = 0;
        window_size = 65535;
    endfunction
    
    virtual function void encode_packet(ref bit [7:0] data[]);
        $display("TCP: Adding sequence number %0d and checksum", sequence_number);
        sequence_number++;
    endfunction
    
    virtual function void decode_packet(ref bit [7:0] data[]);
        $display("TCP: Verifying checksum and sequence number");
    endfunction
    
    virtual function int get_overhead();
        return super.get_overhead() + 4;  // Additional TCP options
    endfunction
endclass

// UDP Protocol
class UDP_Protocol extends Protocol;
    function new();
        super.new("UDP", 8);  // UDP header is 8 bytes
    endfunction
    
    virtual function void encode_packet(ref bit [7:0] data[]);
        $display("UDP: Adding simple header with length and checksum");
    endfunction
    
    virtual function void decode_packet(ref bit [7:0] data[]);
        $display("UDP: Basic header validation");
    endfunction
endclass

// HTTP Protocol (application layer)
class HTTP_Protocol extends Protocol;
    string method;
    string url;
    
    function new(string http_method = "GET", string request_url = "/");
        super.new("HTTP", 0);  // Variable header size
        method = http_method;
        url = request_url;
    endfunction
    
    virtual function void encode_packet(ref bit [7:0] data[]);
        $display("HTTP: Creating %s request for %s", method, url);
    endfunction
    
    virtual function void decode_packet(ref bit [7:0] data[]);
        $display("HTTP: Parsing request/response headers");
    endfunction
    
    virtual function int get_overhead();
        return method.len() + url.len() + 20;  // Estimated header size
    endfunction
endclass
```

### **Example 10:** Simple Drawing Shapes
**Description:** Shape base class with virtual draw() method. Circle, Square, and Triangle override to show different ASCII art representations.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_10__simple_drawing_shapes/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// shape_drawing_system.sv
package shape_drawing_pkg;

  // Base Shape class with virtual draw method
  virtual class Shape;
    protected string shape_name;
    protected int size;
    
    // Constructor
    function new(string name = "Shape", int s = 3);
      shape_name = name;
      size = s;
    endfunction
    
    // Virtual method to be overridden by derived classes
    pure virtual function void draw();
    
    // Get shape information
    function string get_info();
      return $sformatf("%s (size: %0d)", shape_name, size);
    endfunction
  endclass

  // Circle class - draws ASCII circle
  class Circle extends Shape;
    
    function new(int s = 3);
      super.new("Circle", s);
    endfunction
    
    // Override draw method for circle
    virtual function void draw();
      $display("Drawing %s:", get_info());
      case (size)
        1: begin
          $display("  o");
        end
        2: begin
          $display("  ##");
          $display("  ##");
        end
        3: begin
          $display("  ###");
          $display(" #   #");
          $display("  ###");
        end
        default: begin
          $display("  #####");
          $display(" #     #");
          $display("#       #");
          $display(" #     #");
          $display("  #####");
        end
      endcase
      $display();
    endfunction
  endclass

  // Square class - draws ASCII square
  class Square extends Shape;
    
    function new(int s = 3);
      super.new("Square", s);
    endfunction
    
    // Override draw method for square
    virtual function void draw();
      $display("Drawing %s:", get_info());
      for (int i = 0; i < size; i++) begin
        for (int j = 0; j < size; j++) begin
          $write("# ");
        end
        $display();
      end
      $display();
    endfunction
  endclass

  // Triangle class - draws ASCII triangle
  class Triangle extends Shape;
    
    function new(int s = 3);
      super.new("Triangle", s);
    endfunction
    
    // Override draw method for triangle
    virtual function void draw();
      $display("Drawing %s:", get_info());
      for (int i = 0; i < size; i++) begin
        // Add spaces for centering
        for (int j = 0; j < size - i - 1; j++) begin
          $write(" ");
        end
        // Draw triangle pattern
        for (int k = 0; k <= i; k++) begin
          $write("# ");
        end
        $display();
      end
      $display();
    endfunction
  endclass

  // Shape factory class to create different shapes
  class ShapeFactory;
    
    static function Shape create_shape(string shape_type, int size = 3);
      case (shape_type.tolower())
        "circle": begin
          Circle c = new(size);
          return c;
        end
        "square": begin
          Square s = new(size);
          return s;
        end
        "triangle": begin
          Triangle t = new(size);
          return t;
        end
        default: begin
          $display("Warning: Unknown shape type: %s", shape_type);
          return null;
        end
      endcase
    endfunction
  endclass

endpackage

// Design module using the shape drawing system
module shape_drawing_system;
  import shape_drawing_pkg::*;
  
  // Array of shapes for polymorphic demonstration
  Shape shapes[];
  
  initial begin
    $display("=== Simple Drawing Shapes Example ===");
    $display();
    
    // Create array of different shapes
    shapes = new[6];
    shapes[0] = ShapeFactory::create_shape("circle", 2);
    shapes[1] = ShapeFactory::create_shape("square", 3);
    shapes[2] = ShapeFactory::create_shape("triangle", 4);
    shapes[3] = ShapeFactory::create_shape("circle", 3);
    shapes[4] = ShapeFactory::create_shape("square", 2);
    shapes[5] = ShapeFactory::create_shape("triangle", 3);
    
    // Draw all shapes using polymorphism
    foreach (shapes[i]) begin
      if (shapes[i] != null) begin
        shapes[i].draw();
      end
    end
    
    $display("=== All shapes drawn successfully! ===");
    $finish(0);  // Clean exit
  end

endmodule
```

```systemverilog
// shape_drawing_system_testbench.sv
module shape_drawing_testbench;
  import shape_drawing_pkg::*;
  
  // Instantiate design under test
  shape_drawing_system DESIGN_INSTANCE();
  
  // Additional testbench functionality
  Shape test_shape;
  Circle test_circle;
  Square test_square;
  Triangle test_triangle;
  
  initial begin
    // Setup VCD dumping
    $dumpfile("shape_drawing_testbench.vcd");
    $dumpvars(0, shape_drawing_testbench);
    
    #10; // Wait for design to complete
    
    $display("=== Testbench: Individual Shape Testing ===");
    $display();
    
    // Test individual shape creation and drawing
    $display("--- Testing Circle Creation ---");
    test_circle = new(1);
    test_circle.draw();
    
    $display("--- Testing Square Creation ---");
    test_square = new(4);
    test_square.draw();
    
    $display("--- Testing Triangle Creation ---");
    test_triangle = new(5);
    test_triangle.draw();
    
    // Test polymorphic behavior
    $display("--- Testing Polymorphic Behavior ---");
    test_shape = ShapeFactory::create_shape("circle", 3);
    if (test_shape != null) begin
      $display("Shape info: %s", test_shape.get_info());
      test_shape.draw();
    end
    
    test_shape = ShapeFactory::create_shape("square", 2);
    if (test_shape != null) begin
      $display("Shape info: %s", test_shape.get_info());
      test_shape.draw();
    end
    
    test_shape = ShapeFactory::create_shape("triangle", 3);
    if (test_shape != null) begin
      $display("Shape info: %s", test_shape.get_info());
      test_shape.draw();
    end
    
    // Test error handling
    $display("--- Testing Error Handling ---");
    test_shape = ShapeFactory::create_shape("pentagon", 3);
    if (test_shape == null) begin
      $display("Successfully handled unknown shape type");
    end
    
    $display();
    $display("=== Testbench completed successfully! ===");
    $display();
    
    #10; // Small delay before finish
    $finish(0);  // Clean exit
  end

endmodule
```

Verilator Simulation Output:
=== Simple Drawing Shapes Example ===

Drawing Circle (size: 2):
  ##
  ##

Drawing Square (size: 3):
# # #
# # #
# # #

Drawing Triangle (size: 4):
   #
  # #
 # # #
# # # #

Drawing Circle (size: 3):
  ###
 #   #
  ###

Drawing Square (size: 2):
# #
# #

Drawing Triangle (size: 3):
  #
 # #
# # #

=== All shapes drawn successfully! ===
Process finished with return code: 0
Removing Chapter_10_examples/example_10__simple_drawing_shapes/obj_dir directory...
Chapter_10_examples/example_10__simple_drawing_shapes/obj_dir removed successfully.


0

### **Example 11:** Basic File Handlers
**Description:** FileHandler base class with virtual process() method. TextFile, ImageFile, and VideoFile classes provide specific processing logic.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_11__basic_file_handlers/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// file_handler_design.sv

module file_handler_design();
  
  // File processing tasks for different file types
  task process_text_file(string filename);
    $display("Processing text file: %s", filename);
    $display("  - Counting words and lines");
    $display("  - Checking spelling");
    $display("  - Text file processing complete");
  endtask
  
  task process_image_file(string filename);
    $display("Processing image file: %s", filename);
    $display("  - Resizing image");
    $display("  - Applying filters");
    $display("  - Image file processing complete");
  endtask
  
  task process_video_file(string filename);
    $display("Processing video file: %s", filename);
    $display("  - Transcoding video");
    $display("  - Extracting thumbnails");
    $display("  - Video file processing complete");
  endtask
  
  // File handler function that routes to appropriate processor
  function void process_file(string filename, string file_type);
    case (file_type)
      "text": process_text_file(filename);
      "image": process_image_file(filename);
      "video": process_video_file(filename);
      default: $display("Unknown file type: %s", file_type);
    endcase
  endfunction
  
  initial begin
    $display();
    $display("=== File Handler Design Demo ===");
    
    $display("\nProcessing different file types:");
    $display("--------------------------------");
    
    // Process different file types
    process_file("document.txt", "text");
    $display();
    
    process_file("photo.jpg", "image");
    $display();
    
    process_file("movie.mp4", "video");
    $display();
    
    $display("All files processed successfully!");
    $display();
  end

endmodule
```

```systemverilog
// file_handler_design_testbench.sv
module file_handler_testbench;
  
  // Instantiate design under test
  file_handler_design DESIGN_INSTANCE();
  
  // Test file processing tasks
  task test_file_processing();
    $display("=== Testbench Additional Tests ===");
    $display("\nTesting individual file handlers:");
    $display("---------------------------------");
    
    // Test text file processing
    $display("Testing text file handler:");
    DESIGN_INSTANCE.process_text_file("readme.txt");
    $display();
    
    // Test image file processing
    $display("Testing image file handler:");
    DESIGN_INSTANCE.process_image_file("screenshot.png");
    $display();
    
    // Test video file processing
    $display("Testing video file handler:");
    DESIGN_INSTANCE.process_video_file("tutorial.avi");
    $display();
    
    // Test file handler function with different types
    $display("Testing file handler function:");
    DESIGN_INSTANCE.process_file("config.txt", "text");
    $display();
    DESIGN_INSTANCE.process_file("banner.gif", "image");
    $display();
    DESIGN_INSTANCE.process_file("demo.mov", "video");
    $display();
    
    // Test unknown file type
    $display("Testing unknown file type:");
    DESIGN_INSTANCE.process_file("data.bin", "binary");
    $display();
  endtask
  
  initial begin
    // Setup waveform dumping
    $dumpfile("file_handler_testbench.vcd");
    $dumpvars(0, file_handler_testbench);
    
    #1; // Wait for design to initialize
    
    // Run additional tests
    test_file_processing();
    
    $display("Testbench completed successfully!");
    $display();
    
    #10; // Final delay
    $finish;
  end

endmodule
```

Verilator Simulation Output:

=== File Handler Design Demo ===

Processing different file types:
--------------------------------
Processing text file: document.txt
  - Counting words and lines
  - Checking spelling
  - Text file processing complete

Processing image file: photo.jpg
  - Resizing image
  - Applying filters
  - Image file processing complete

Processing video file: movie.mp4
  - Transcoding video
  - Extracting thumbnails
  - Video file processing complete

All files processed successfully!

=== Testbench Additional Tests ===

Testing individual file handlers:
---------------------------------
Testing text file handler:
Processing text file: readme.txt
  - Counting words and lines
  - Checking spelling
  - Text file processing complete

Testing image file handler:
Processing image file: screenshot.png
  - Resizing image
  - Applying filters
  - Image file processing complete

Testing video file handler:
Processing video file: tutorial.avi
  - Transcoding video
  - Extrac

0

### **Example 12:** Simple Game Units
**Description:** Unit base class with virtual attack() method. Archer, Knight, and Wizard override to show different attack patterns and damage.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_12__simple_game_units/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// game_units_design.sv
`include "game_units_package.sv"

module game_units_design();
  import game_units_pkg::*;
  
  // Declare unit handles
  GameUnit battle_units[];
  Archer archer_hero;
  Knight knight_hero;
  Wizard wizard_hero;
  
  initial begin
    $display("=== Game Units Battle System ===");
    $display();
    
    // Create different unit types
    archer_hero = new("Legolas");
    knight_hero = new("Sir Galahad");
    wizard_hero = new("Gandalf");
    
    // Create array of units for polymorphic behavior
    battle_units = new[3];
    battle_units[0] = archer_hero;
    battle_units[1] = knight_hero;
    battle_units[2] = wizard_hero;
    
    // Display unit information
    $display("Battle Participants:");
    foreach (battle_units[i]) begin
      $display("  %0d. %s", i+1, battle_units[i].get_info());
    end
    $display();
    
    // Demonstrate polymorphic attack behavior
    $display("=== Battle Begins! ===");
    foreach (battle_units[i]) begin
      $display("Turn %0d:", i+1);
      battle_units[i].attack();  // Calls overridden method
      $display();
    end
    
    // Demonstrate damage system
    $display("=== Damage Demo ===");
    $display("Knight takes damage from Wizard's spell:");
    knight_hero.take_damage(20);
    $display();
    
    $display("Archer takes critical damage:");
    archer_hero.take_damage(85);
    $display();
    
    $display("Final Status:");
    foreach (battle_units[i]) begin
      $display("  %s", battle_units[i].get_info());
    end
    
  end
  
endmodule
```

```systemverilog
// game_units_package.sv
package game_units_pkg;

  // Base Game Unit class with virtual attack method
  virtual class GameUnit;
    protected string unit_name;
    protected int health;
    protected int damage;
    
    function new(string name = "Unknown", int hp = 100, int dmg = 10);
      this.unit_name = name;
      this.health = hp;
      this.damage = dmg;
    endfunction
    
    // Virtual attack method to be overridden by derived classes
    virtual function void attack();
      $display("%s performs a basic attack for %0d damage!", 
               unit_name, damage);
    endfunction
    
    // Get unit information
    function string get_info();
      return $sformatf("%s (HP: %0d, DMG: %0d)", 
                       unit_name, health, damage);
    endfunction
    
    // Take damage
    function void take_damage(int dmg);
      health -= dmg;
      if (health <= 0) begin
        health = 0;
        $display("%s has been defeated!", unit_name);
      end else begin
        $display("%s takes %0d damage! Health: %0d", 
                 unit_name, dmg, health);
      end
    endfunction
    
  endclass

  // Archer class - ranged attacks with precision
  class Archer extends GameUnit;
    
    function new(string name = "Archer");
      super.new(name, 80, 15);  // Lower health, higher damage
    endfunction
    
    // Override attack method
    virtual function void attack();
      $display("%s draws bow and fires a precise arrow for %0d damage!", 
               unit_name, damage);
      $display("  *Swoosh* -> Direct hit!");
    endfunction
    
  endclass

  // Knight class - melee attacks with defense
  class Knight extends GameUnit;
    
    function new(string name = "Knight");
      super.new(name, 120, 12);  // Higher health, moderate damage
    endfunction
    
    // Override attack method
    virtual function void attack();
      $display("%s raises sword and strikes with %0d damage!", 
               unit_name, damage);
      $display("  *Clang* -> Steel meets steel!");
    endfunction
    
  endclass

  // Wizard class - magical attacks with special effects
  class Wizard extends GameUnit;
    
    function new(string name = "Wizard");
      super.new(name, 70, 20);  // Lowest health, highest damage
    endfunction
    
    // Override attack method
    virtual function void attack();
      $display("%s casts a spell dealing %0d magical damage!", 
               unit_name, damage);
      $display("  *Zap* -> Arcane energy crackles!");
    endfunction
    
  endclass

endpackage
```

```systemverilog
// game_units_testbench.sv
module game_units_testbench;
  import game_units_pkg::*;
  
  // Testbench variables
  GameUnit test_units[];
  Archer test_archer;
  Knight test_knight;
  Wizard test_wizard;
  
  // Instantiate design under test
  game_units_design DUT();
  
  initial begin
    // Dump waves for debugging
    $dumpfile("game_units_testbench.vcd");
    $dumpvars(0, game_units_testbench);
    
    #1;  // Wait for design to complete
    
    $display("=== Testbench: Additional Unit Tests ===");
    $display();
    
    // Test individual unit creation and methods
    test_archer = new("Robin Hood");
    test_knight = new("Sir Lancelot");
    test_wizard = new("Merlin");
    
    $display("Testing individual unit attacks:");
    $display("Archer:");
    test_archer.attack();
    $display();
    
    $display("Knight:");
    test_knight.attack();
    $display();
    
    $display("Wizard:");
    test_wizard.attack();
    $display();
    
    // Test polymorphic array behavior
    test_units = new[3];
    test_units[0] = test_archer;
    test_units[1] = test_knight;
    test_units[2] = test_wizard;
    
    $display("Testing polymorphic behavior:");
    foreach (test_units[i]) begin
      $display("Unit %0d attack:", i+1);
      test_units[i].attack();
    end
    $display();
    
    // Test damage system with different scenarios
    $display("Testing damage system:");
    $display("Before damage: %s", test_knight.get_info());
    test_knight.take_damage(30);
    $display("After 30 damage: %s", test_knight.get_info());
    test_knight.take_damage(100);  // Should defeat the knight
    $display("After 100 damage: %s", test_knight.get_info());
    $display();
    
    $display("=== All Tests Complete ===");
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Game Units Battle System ===

Battle Participants:
  1. Legolas (HP: 80, DMG: 15)
  2. Sir Galahad (HP: 120, DMG: 12)
  3. Gandalf (HP: 70, DMG: 20)

=== Battle Begins! ===
Turn 1:
Legolas draws bow and fires a precise arrow for 15 damage!
  *Swoosh* -> Direct hit!

Turn 2:
Sir Galahad raises sword and strikes with 12 damage!
  *Clang* -> Steel meets steel!

Turn 3:
Gandalf casts a spell dealing 20 magical damage!
  *Zap* -> Arcane energy crackles!

=== Damage Demo ===
Knight takes damage from Wizard's spell:
Sir Galahad takes 20 damage! Health: 100

Archer takes critical damage:
Legolas has been defeated!

Final Status:
  Legolas (HP: 0, DMG: 15)
  Sir Galahad (HP: 100, DMG: 12)
  Gandalf (HP: 70, DMG: 20)
=== Testbench: Additional Unit Tests ===

Testing individual unit attacks:
Archer:
Robin Hood draws bow and fires a precise arrow for 15 damage!
  *Swoosh* -> Direct hit!

Knight:
Sir Lancelot raises sword and strikes with 12 damage!
  *Clang* -> Ste

0

## Abstract Classes

While SystemVerilog doesn't have explicit abstract class syntax, you can create abstract-like behavior using pure virtual methods and base classes that shouldn't be instantiated directly.

### Abstract Class Pattern

```systemverilog
// Abstract Database Connection class
class DatabaseConnection;
    string connection_string;
    bit connected;
    
    function new(string conn_str);
        connection_string = conn_str;
        connected = 0;
    endfunction
    
    // Pure virtual methods (must be implemented by derived classes)
    pure virtual function bit connect();
    pure virtual function void disconnect();
    pure virtual function string execute_query(string query);
    pure virtual function bit is_connected();
    
    // Concrete method that can be inherited
    function void log_operation(string operation);
        $display("[%0t] Database Operation: %s", $time, operation);
    endfunction
endclass

// MySQL Database implementation
class MySQLConnection extends DatabaseConnection;
    int port;
    string database_name;
    
    function new(string host, int p = 3306, string db = "test");
        super.new($sformatf("mysql://%s:%0d/%s", host, p, db));
        port = p;
        database_name = db;
    endfunction
    
    // Implement abstract methods
    virtual function bit connect();
        log_operation("Connecting to MySQL");
        connected = 1;
        $display("Connected to MySQL database: %s", database_name);
        return connected;
    endfunction
    
    virtual function void disconnect();
        if (connected) begin
            log_operation("Disconnecting from MySQL");
            connected = 0;
            $display("Disconnected from MySQL");
        end
    endfunction
    
    virtual function string execute_query(string query);
        if (!connected) begin
            $display("Error: Not connected to database");
            return "";
        end
        log_operation($sformatf("Executing: %s", query));
        return "MySQL query result";
    endfunction
    
    virtual function bit is_connected();
        return connected;
    endfunction
endclass

// PostgreSQL Database implementation
class PostgreSQLConnection extends DatabaseConnection;
    string schema_name;
    
    function new(string host, string schema = "public");
        super.new($sformatf("postgresql://%s/%s", host, schema));
        schema_name = schema;
    endfunction
    
    virtual function bit connect();
        log_operation("Connecting to PostgreSQL");
        connected = 1;
        $display("Connected to PostgreSQL schema: %s", schema_name);
        return connected;
    endfunction
    
    virtual function void disconnect();
        if (connected) begin
            log_operation("Disconnecting from PostgreSQL");
            connected = 0;
            $display("Disconnected from PostgreSQL");
        end
    endfunction
    
    virtual function string execute_query(string query);
        if (!connected) begin
            $display("Error: Not connected to database");
            return "";
        end
        log_operation($sformatf("Executing on schema %s: %s", schema_name, query));
        return "PostgreSQL query result";
    endfunction
    
    virtual function bit is_connected();
        return connected;
    endfunction
endclass
```

### **Example 13:** Basic Database Operations
**Description:** Database base class with pure virtual connect(), query(), and disconnect() methods. MySQL and SQLite classes implement these methods.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_13__basic_database_operations/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// database_operations.sv
// Chapter 10 Example 13: Basic Database Operations

package database_pkg;

  // Base database class with pure virtual methods
  virtual class database_base;
    protected string connection_string;
    protected bit is_connected;
    
    function new(string conn_str);
      this.connection_string = conn_str;
      this.is_connected = 0;
    endfunction
    
    // Pure virtual methods that must be implemented by derived classes
    pure virtual function bit connect();
    pure virtual function string query(string sql_command);
    pure virtual function void disconnect();
    
    // Common method for all database types
    function bit is_connection_active();
      return this.is_connected;
    endfunction
    
    function string get_connection_string();
      return this.connection_string;
    endfunction
  endclass

  // MySQL database implementation
  class mysql_database extends database_base;
    
    function new(string conn_str);
      super.new(conn_str);
    endfunction
    
    virtual function bit connect();
      $display("[MySQL] Connecting to database: %s", connection_string);
      this.is_connected = 1;
      $display("[MySQL] Connection established successfully");
      return 1;
    endfunction
    
    virtual function string query(string sql_command);
      string result;
      if (!is_connected) begin
        $display("[MySQL] Error: Not connected to database");
        return "ERROR: Not connected";
      end
      
      $display("[MySQL] Executing query: %s", sql_command);
      
      // Simulate different query results
      if (sql_command.substr(0, 5) == "SELECT") begin
        result = "MySQL Result: Found 3 rows";
      end else if (sql_command.substr(0, 5) == "INSERT") begin
        result = "MySQL Result: 1 row inserted";
      end else if (sql_command.substr(0, 5) == "UPDATE") begin
        result = "MySQL Result: 2 rows updated";
      end else begin
        result = "MySQL Result: Query executed";
      end
      
      $display("[MySQL] Query result: %s", result);
      return result;
    endfunction
    
    virtual function void disconnect();
      if (is_connected) begin
        $display("[MySQL] Disconnecting from database");
        this.is_connected = 0;
        $display("[MySQL] Disconnected successfully");
      end else begin
        $display("[MySQL] Already disconnected");
      end
    endfunction
  endclass

  // SQLite database implementation
  class sqlite_database extends database_base;
    
    function new(string conn_str);
      super.new(conn_str);
    endfunction
    
    virtual function bit connect();
      $display("[SQLite] Opening database file: %s", connection_string);
      this.is_connected = 1;
      $display("[SQLite] Database file opened successfully");
      return 1;
    endfunction
    
    virtual function string query(string sql_command);
      string result;
      if (!is_connected) begin
        $display("[SQLite] Error: Database file not opened");
        return "ERROR: Not connected";
      end
      
      $display("[SQLite] Executing query: %s", sql_command);
      
      // Simulate different query results
      if (sql_command.substr(0, 5) == "SELECT") begin
        result = "SQLite Result: Found 5 rows";
      end else if (sql_command.substr(0, 5) == "INSERT") begin
        result = "SQLite Result: 1 row inserted";
      end else if (sql_command.substr(0, 5) == "DELETE") begin
        result = "SQLite Result: 1 row deleted";
      end else begin
        result = "SQLite Result: Query executed";
      end
      
      $display("[SQLite] Query result: %s", result);
      return result;
    endfunction
    
    virtual function void disconnect();
      if (is_connected) begin
        $display("[SQLite] Closing database file");
        this.is_connected = 0;
        $display("[SQLite] Database file closed successfully");
      end else begin
        $display("[SQLite] Database file already closed");
      end
    endfunction
  endclass

endpackage

module database_operations;
  import database_pkg::*;
  
  initial begin
    $display("=== Database Operations Example ===");
    $display();
  end
  
endmodule
```

```systemverilog
// database_operations_testbench.sv
// Testbench for Chapter 10 Example 13: Basic Database Operations

module database_testbench;
  import database_pkg::*;
  
  // Instantiate the design under test
  database_operations DUT();
  
  // Test variables
  mysql_database mysql_db;
  sqlite_database sqlite_db;
  database_base db_handle;
  string query_result;
  bit connection_result;
  
  initial begin
    // Setup waveform dumping
    $dumpfile("database_testbench.vcd");
    $dumpvars(0, database_testbench);
    
    $display("=== Testing Database Operations ===");
    $display();
    
    // Test MySQL database
    $display("--- Testing MySQL Database ---");
    mysql_db = new("server=localhost;database=test;user=admin");
    
    // Test connection
    connection_result = mysql_db.connect();
    if (connection_result) begin
      $display("MySQL connection status: %s", 
               mysql_db.is_connection_active() ? "ACTIVE" : "INACTIVE");
    end
    
    // Test queries
    query_result = mysql_db.query("SELECT * FROM users");
    query_result = mysql_db.query("INSERT INTO users VALUES (1, 'John')");
    query_result = mysql_db.query("UPDATE users SET name='Jane' WHERE id=1");
    
    // Test disconnection
    mysql_db.disconnect();
    $display("MySQL connection status after disconnect: %s", 
             mysql_db.is_connection_active() ? "ACTIVE" : "INACTIVE");
    
    $display();
    
    // Test SQLite database
    $display("--- Testing SQLite Database ---");
    sqlite_db = new("./test_database.db");
    
    // Test connection
    connection_result = sqlite_db.connect();
    if (connection_result) begin
      $display("SQLite connection status: %s", 
               sqlite_db.is_connection_active() ? "ACTIVE" : "INACTIVE");
    end
    
    // Test queries
    query_result = sqlite_db.query("SELECT * FROM products");
    query_result = sqlite_db.query("INSERT INTO products VALUES (1, 'Widget')");
    query_result = sqlite_db.query("DELETE FROM products WHERE id=1");
    
    // Test disconnection
    sqlite_db.disconnect();
    $display("SQLite connection status after disconnect: %s", 
             sqlite_db.is_connection_active() ? "ACTIVE" : "INACTIVE");
    
    $display();
    
    // Test polymorphism - using base class handle
    $display("--- Testing Polymorphism ---");
    
    // Point to MySQL database
    db_handle = mysql_db;
    $display("Using base class handle with MySQL:");
    connection_result = db_handle.connect();
    query_result = db_handle.query("SELECT COUNT(*) FROM orders");
    db_handle.disconnect();
    
    $display();
    
    // Point to SQLite database
    db_handle = sqlite_db;
    $display("Using base class handle with SQLite:");
    connection_result = db_handle.connect();
    query_result = db_handle.query("SELECT COUNT(*) FROM inventory");
    db_handle.disconnect();
    
    $display();
    
    // Test error handling - query without connection
    $display("--- Testing Error Handling ---");
    query_result = mysql_db.query("SELECT * FROM test_table");
    query_result = sqlite_db.query("SELECT * FROM test_table");
    
    $display();
    $display("=== Database Operations Test Complete ===");
    
    #10;
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Database Operations Example ===

=== Testing Database Operations ===

--- Testing MySQL Database ---
[MySQL] Connecting to database: server=localhost;database=test;user=admin
[MySQL] Connection established successfully
MySQL connection status:   ACTIVE
[MySQL] Executing query: SELECT * FROM users
[MySQL] Query result: MySQL Result: Found 3 rows
[MySQL] Executing query: INSERT INTO users VALUES (1, 'John')
[MySQL] Query result: MySQL Result: 1 row inserted
[MySQL] Executing query: UPDATE users SET name='Jane' WHERE id=1
[MySQL] Query result: MySQL Result: 2 rows updated
[MySQL] Disconnecting from database
[MySQL] Disconnected successfully
MySQL connection status after disconnect: INACTIVE

--- Testing SQLite Database ---
[SQLite] Opening database file: ./test_database.db
[SQLite] Database file opened successfully
SQLite connection status:   ACTIVE
[SQLite] Executing query: SELECT * FROM products
[SQLite] Query result: SQLite Result: Found 5 rows
[SQLite]

0

### **Example 14:** Simple Printer Interface
**Description:** Printer base class with pure virtual print() method. LaserPrinter and InkjetPrinter implement specific printing mechanisms.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_14__simple_printer_interface/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// printer_interface_design.sv
package printer_interface_pkg;

  // Base printer class with pure virtual print method
  virtual class BasePrinter;
    string printer_name;
    
    function new(string name);
      this.printer_name = name;
    endfunction
    
    // Pure virtual method - must be implemented by derived classes
    pure virtual function void print(string document);
    
    // Common method for all printers
    function void display_status();
      $display("[%s] Printer is ready", printer_name);
    endfunction
  endclass

  // Laser printer implementation
  class LaserPrinter extends BasePrinter;
    int toner_level;
    
    function new(string name, int initial_toner = 100);
      super.new(name);
      this.toner_level = initial_toner;
    endfunction
    
    // Implementation of pure virtual method
    virtual function void print(string document);
      if (toner_level > 0) begin
        $display("[%s] Laser printing: '%s'", printer_name, document);
        $display("[%s] Using laser technology for crisp text", printer_name);
        toner_level -= 5;
        $display("[%s] Toner level: %0d%%", printer_name, toner_level);
      end else begin
        $display("[%s] ERROR: Toner empty!", printer_name);
      end
    endfunction
    
    function void replace_toner();
      toner_level = 100;
      $display("[%s] Toner cartridge replaced", printer_name);
    endfunction
  endclass

  // Inkjet printer implementation
  class InkjetPrinter extends BasePrinter;
    int ink_level;
    
    function new(string name, int initial_ink = 80);
      super.new(name);
      this.ink_level = initial_ink;
    endfunction
    
    // Implementation of pure virtual method
    virtual function void print(string document);
      if (ink_level > 0) begin
        $display("[%s] Inkjet printing: '%s'", printer_name, document);
        $display("[%s] Using liquid ink for vibrant colors", printer_name);
        ink_level -= 10;
        $display("[%s] Ink level: %0d%%", printer_name, ink_level);
      end else begin
        $display("[%s] ERROR: Ink cartridge empty!", printer_name);
      end
    endfunction
    
    function void refill_ink();
      ink_level = 80;
      $display("[%s] Ink cartridge refilled", printer_name);
    endfunction
  endclass

endpackage

module printer_interface_design_module;
  // This module serves as the design under test container
  // The actual functionality is in the package classes
  
  initial begin
    $display("=== Printer Interface Design Module ===");
    $display("Classes defined in printer_interface_pkg package");
    $display("Ready for testbench instantiation");
  end
  
endmodule
```

```systemverilog
// printer_interface_design_testbench.sv
module printer_interface_testbench;
  import printer_interface_pkg::*;
  
  // Instantiate design under test
  printer_interface_design_module DUT();
  
  // Test variables
  LaserPrinter laser_printer;
  InkjetPrinter inkjet_printer;
  BasePrinter printer_array[2];
  
  initial begin
    // Configure VCD dumping
    $dumpfile("printer_interface_testbench.vcd");
    $dumpvars(0, printer_interface_testbench);
    
    $display("\n=== Printer Interface Testbench ===");
    $display("Testing inheritance and polymorphism with printers");
    
    // Create printer instances
    laser_printer = new("HP LaserJet Pro", 75);
    inkjet_printer = new("Canon PIXMA", 60);
    
    $display("\n--- Individual Printer Testing ---");
    
    // Test laser printer
    $display("\n1. Testing Laser Printer:");
    laser_printer.display_status();
    laser_printer.print("Business Report");
    laser_printer.print("Invoice #12345");
    
    // Test inkjet printer
    $display("\n2. Testing Inkjet Printer:");
    inkjet_printer.display_status();
    inkjet_printer.print("Family Photo");
    inkjet_printer.print("Color Brochure");
    
    $display("\n--- Polymorphism Demonstration ---");
    
    // Demonstrate polymorphism using base class handles
    printer_array[0] = laser_printer;
    printer_array[1] = inkjet_printer;
    
    $display("\n3. Polymorphic printing:");
    for (int i = 0; i < 2; i++) begin
      $display("\nPrinter %0d:", i+1);
      printer_array[i].display_status();
      printer_array[i].print($sformatf("Document_%0d", i+1));
    end
    
    $display("\n--- Resource Management Testing ---");
    
    // Test resource depletion and refill
    $display("\n4. Testing resource management:");
    
    // Deplete laser printer toner
    repeat(15) begin
      laser_printer.print("Test Page");
    end
    
    // Refill toner
    laser_printer.replace_toner();
    laser_printer.print("After Toner Replacement");
    
    // Deplete inkjet printer ink
    repeat(6) begin
      inkjet_printer.print("Test Image");
    end
    
    // Refill ink
    inkjet_printer.refill_ink();
    inkjet_printer.print("After Ink Refill");
    
    $display("\n--- Final Status Check ---");
    $display("\n5. Final printer status:");
    laser_printer.display_status();
    inkjet_printer.display_status();
    
    #10;
    $display("\n=== Printer Interface Test Complete ===");
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Printer Interface Design Module ===
Classes defined in printer_interface_pkg package
Ready for testbench instantiation

=== Printer Interface Testbench ===
Testing inheritance and polymorphism with printers

--- Individual Printer Testing ---

1. Testing Laser Printer:
[HP LaserJet Pro] Printer is ready
[HP LaserJet Pro] Laser printing: 'Business Report'
[HP LaserJet Pro] Using laser technology for crisp text
[HP LaserJet Pro] Toner level: 70%
[HP LaserJet Pro] Laser printing: 'Invoice #12345'
[HP LaserJet Pro] Using laser technology for crisp text
[HP LaserJet Pro] Toner level: 65%

2. Testing Inkjet Printer:
[Canon PIXMA] Printer is ready
[Canon PIXMA] Inkjet printing: 'Family Photo'
[Canon PIXMA] Using liquid ink for vibrant colors
[Canon PIXMA] Ink level: 50%
[Canon PIXMA] Inkjet printing: 'Color Brochure'
[Canon PIXMA] Using liquid ink for vibrant colors
[Canon PIXMA] Ink level: 40%

--- Polymorphism Demonstration ---

3. Polymorphic printing:

Pri

0

### **Example 15:** Basic Sensor Interface
**Description:** Sensor base class with pure virtual read_value() method. TemperatureSensor and HumiditySensor implement specific reading logic.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_15__basic_sensor_interface/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// sensor_interface_design.sv
package sensor_interface_pkg;

  // Base sensor class with pure virtual method
  virtual class BaseSensor;
    protected string sensor_name;
    protected int sensor_id;
    
    // Constructor
    function new(string name = "Unknown", int id = 0);
      this.sensor_name = name;
      this.sensor_id = id;
    endfunction
    
    // Pure virtual method - must be implemented by derived classes
    pure virtual function real read_value();
    
    // Common method for all sensors
    virtual function string get_sensor_info();
      return $sformatf("Sensor: %s (ID: %0d)", sensor_name, sensor_id);
    endfunction
    
    // Display sensor reading with formatting
    virtual function void display_reading();
      real value = read_value();
      $display("%s - Reading: %0.2f", get_sensor_info(), value);
    endfunction
    
  endclass

  // Temperature sensor implementation
  class TemperatureSensor extends BaseSensor;
    local real temperature_celsius;
    
    // Constructor
    function new(string name = "Temperature", int id = 0, 
                 real initial_temp = 25.0);
      super.new(name, id);
      this.temperature_celsius = initial_temp;
    endfunction
    
    // Implement pure virtual method
    virtual function real read_value();
      // Simulate temperature reading with some variation
      int rand_val = $random % 10;
      real variation = (rand_val - 5) * 0.1;
      temperature_celsius = temperature_celsius + variation;
      return temperature_celsius;
    endfunction
    
    // Temperature-specific method
    function real get_fahrenheit();
      return (read_value() * 9.0/5.0) + 32.0;
    endfunction
    
    // Override display method for temperature-specific formatting
    virtual function void display_reading();
      real celsius = read_value();
      real fahrenheit = (celsius * 9.0/5.0) + 32.0;
      $display("%s - Temperature: %0.2fÂ°C (%0.2fÂ°F)", 
               get_sensor_info(), celsius, fahrenheit);
    endfunction
    
  endclass

  // Humidity sensor implementation
  class HumiditySensor extends BaseSensor;
    local real humidity_percent;
    
    // Constructor
    function new(string name = "Humidity", int id = 0, 
                 real initial_humidity = 50.0);
      super.new(name, id);
      this.humidity_percent = initial_humidity;
    endfunction
    
    // Implement pure virtual method
    virtual function real read_value();
      // Simulate humidity reading with some variation
      int rand_val = $random % 20;
      real variation = (rand_val - 10) * 0.1;
      humidity_percent = humidity_percent + variation;
      // Clamp to valid range
      if (humidity_percent < 0.0) humidity_percent = 0.0;
      if (humidity_percent > 100.0) humidity_percent = 100.0;
      return humidity_percent;
    endfunction
    
    // Humidity-specific method
    function string get_humidity_level();
      real humidity = read_value();
      if (humidity < 30.0) return "Low";
      else if (humidity < 60.0) return "Normal";
      else return "High";
    endfunction
    
    // Override display method for humidity-specific formatting
    virtual function void display_reading();
      real humidity = read_value();
      string level = get_humidity_level();
      $display("%s - Humidity: %0.2f%% (%s)", 
               get_sensor_info(), humidity, level);
    endfunction
    
  endclass

endpackage

// Design module using the sensor interface
module sensor_interface_module;
  import sensor_interface_pkg::*;
  
  // Sensor instances
  TemperatureSensor temp_sensor_indoor;
  TemperatureSensor temp_sensor_outdoor;
  HumiditySensor humidity_sensor;
  
  // Array of base sensor handles for polymorphism
  BaseSensor sensor_array[];
  
  initial begin
    $display("=== Basic Sensor Interface Example ===");
    $display();
    
    // Create sensor instances
    temp_sensor_indoor = new("Indoor Temp", 1, 22.5);
    temp_sensor_outdoor = new("Outdoor Temp", 2, 15.0);
    humidity_sensor = new("Room Humidity", 3, 45.0);
    
    // Setup sensor array for polymorphic access
    sensor_array = new[3];
    sensor_array[0] = temp_sensor_indoor;
    sensor_array[1] = temp_sensor_outdoor;
    sensor_array[2] = humidity_sensor;
    
    $display("Initial sensor readings:");
    foreach(sensor_array[i]) begin
      sensor_array[i].display_reading();
    end
    
    $display();
    $display("Sensor readings after simulation time:");
    
    // Simulate some time passing and read sensors again
    for (int cycle = 0; cycle < 3; cycle++) begin
      $display("--- Cycle %0d ---", cycle + 1);
      foreach(sensor_array[i]) begin
        sensor_array[i].display_reading();
      end
      $display();
    end
    
    // Demonstrate specific sensor methods
    $display("=== Specific Sensor Features ===");
    $display("Indoor temperature in Fahrenheit: %0.2fÂ°F", 
             temp_sensor_indoor.get_fahrenheit());
    $display("Humidity level: %s", humidity_sensor.get_humidity_level());
    
    $display();
    $display("=== Sensor Information ===");
    foreach(sensor_array[i]) begin
      $display("Sensor %0d: %s", i, sensor_array[i].get_sensor_info());
    end
    
  end
  
endmodule
```

```systemverilog
// sensor_interface_design_testbench.sv
module sensor_interface_testbench;
  import sensor_interface_pkg::*;
  
  // Instantiate the design under test
  sensor_interface_module SENSOR_INTERFACE_INSTANCE();
  
  // Test-specific sensor instances
  TemperatureSensor test_temp_sensor;
  HumiditySensor test_humidity_sensor;
  BaseSensor polymorphic_sensor;
  
  initial begin
    // Setup VCD dumping
    $dumpfile("sensor_interface_testbench.vcd");
    $dumpvars(0, sensor_interface_testbench);
    
    #1; // Wait for design to initialize
    
    $display("=== Testbench: Basic Sensor Interface ===");
    $display();
    
    // Test 1: Create and test individual sensors
    $display("TEST 1: Individual sensor creation and testing");
    test_temp_sensor = new("Test Temperature", 100, 20.0);
    test_humidity_sensor = new("Test Humidity", 101, 60.0);
    
    $display("Created sensors:");
    test_temp_sensor.display_reading();
    test_humidity_sensor.display_reading();
    
    $display();
    
    // Test 2: Polymorphic behavior
    $display("TEST 2: Polymorphic behavior");
    polymorphic_sensor = test_temp_sensor;
    $display("Polymorphic access to temperature sensor:");
    polymorphic_sensor.display_reading();
    
    polymorphic_sensor = test_humidity_sensor;
    $display("Polymorphic access to humidity sensor:");
    polymorphic_sensor.display_reading();
    
    $display();
    
    // Test 3: Multiple readings to show variation
    $display("TEST 3: Multiple readings showing sensor variation");
    for (int i = 0; i < 5; i++) begin
      $display("Reading %0d:", i + 1);
      $display("  Temperature: %0.2fÂ°C", test_temp_sensor.read_value());
      $display("  Humidity: %0.2f%%", test_humidity_sensor.read_value());
    end
    
    $display();
    
    // Test 4: Specific sensor methods
    $display("TEST 4: Sensor-specific methods");
    $display("Temperature in Fahrenheit: %0.2fÂ°F", 
             test_temp_sensor.get_fahrenheit());
    $display("Humidity level: %s", test_humidity_sensor.get_humidity_level());
    
    $display();
    
    // Test 5: Boundary conditions for humidity
    $display("TEST 5: Humidity boundary conditions");
    test_humidity_sensor = new("Boundary Test", 102, 95.0);
    for (int i = 0; i < 10; i++) begin
      real humidity = test_humidity_sensor.read_value();
      $display("Humidity reading %0d: %0.2f%%", i + 1, humidity);
    end
    
    $display();
    $display("=== Testbench Complete ===");
    
    #10; // Final delay
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:
=== Basic Sensor Interface Example ===

Initial sensor readings:
Sensor: Indoor Temp (ID: 1) - Temperature: 22.00Â°C (71.60Â°F)
Sensor: Outdoor Temp (ID: 2) - Temperature: 14.50Â°C (58.10Â°F)
Sensor: Room Humidity (ID: 3) - Humidity: 43.00% (Normal)

Sensor readings after simulation time:
--- Cycle 1 ---
Sensor: Indoor Temp (ID: 1) - Temperature: 20.90Â°C (69.62Â°F)
Sensor: Outdoor Temp (ID: 2) - Temperature: 13.40Â°C (56.12Â°F)
Sensor: Room Humidity (ID: 3) - Humidity: 40.80% (Normal)

--- Cycle 2 ---
Sensor: Indoor Temp (ID: 1) - Temperature: 21.10Â°C (69.98Â°F)
Sensor: Outdoor Temp (ID: 2) - Temperature: 12.20Â°C (53.96Â°F)
Sensor: Room Humidity (ID: 3) - Humidity: 39.30% (Normal)

--- Cycle 3 ---
Sensor: Indoor Temp (ID: 1) - Temperature: 20.50Â°C (68.90Â°F)
Sensor: Outdoor Temp (ID: 2) - Temperature: 12.10Â°C (53.78Â°F)
Sensor: Room Humidity (ID: 3) - Humidity: 33.60% (Normal)

=== Specific Sensor Features ===
Indoor temperature in Fahrenheit: 66.56Â°F

0

## Polymorphism Examples

Polymorphism allows objects of different types to be treated uniformly through a common interface, with method calls resolved at runtime based on the actual object type.

### Example 1: Graphics Rendering System

```systemverilog
// Test module demonstrating polymorphism with shapes
module polymorphism_demo;
    
    // Array of shape handles (polymorphic collection)
    Shape shapes[];
    Rectangle rect;
    Circle circ;
    Triangle tri;
    
    initial begin
        // Create different shape objects
        rect = new(5.0, 3.0);
        circ = new(2.5);
        tri = new(4.0, 6.0);
        
        // Store them in polymorphic array
        shapes = new[3];
        shapes[0] = rect;  // Rectangle assigned to Shape handle
        shapes[1] = circ;  // Circle assigned to Shape handle
        shapes[2] = tri;   // Triangle assigned to Shape handle
        
        $display("=== Polymorphic Shape Processing ===");
        
        // Process all shapes polymorphically
        foreach (shapes[i]) begin
            $display("\nShape %0d:", i+1);
            shapes[i].draw();  // Calls appropriate draw method
            $display("Area: %0.2f", shapes[i].calculate_area());  // Calls appropriate calculate_area
        end
        
        // Calculate total area
        real total_area = 0.0;
        foreach (shapes[i]) begin
            total_area += shapes[i].calculate_area();
        end
        $display("\nTotal area of all shapes: %0.2f", total_area);
    end
endmodule
```

### Example 2: Network Protocol Handler

```systemverilog
// Protocol handler demonstrating polymorphism
class NetworkStack;
    Protocol protocols[];
    
    function new();
        protocols = new[3];
        protocols[0] = new TCP_Protocol();
        protocols[1] = new UDP_Protocol();
        protocols[2] = new HTTP_Protocol("POST", "/api/data");
    endfunction
    
    // Process packet through all protocol layers
    function void process_packet(ref bit [7:0] data[]);
        $display("=== Processing Packet Through Network Stack ===");
        
        foreach (protocols[i]) begin
            $display("\n--- Layer %0d ---", i+1);
            protocols[i].display_info();
            protocols[i].encode_packet(data);
            $display("Overhead: %0d bytes", protocols[i].get_overhead());
        end
        
        $display("\n=== Decoding Packet ===");
        // Decode in reverse order
        for (int i = protocols.size()-1; i >= 0; i--) begin
            $display("\n--- Layer %0d Decode ---", i+1);
            protocols[i].decode_packet(data);
        end
    end
    
    function int calculate_total_overhead();
        int total = 0;
        foreach (protocols[i]) begin
            total += protocols[i].get_overhead();
        end
        return total;
    endfunction
endclass

// Test module for network protocols
module network_demo;
    NetworkStack stack;
    bit [7:0] packet_data[];
    
    initial begin
        stack = new();
        packet_data = new[100];  // 100-byte data packet
        
        // Initialize packet with dummy data
        foreach (packet_data[i]) begin
            packet_data[i] = i % 256;
        end
        
        // Process packet through protocol stack
        stack.process_packet(packet_data);
        
        $display("\nTotal Protocol Overhead: %0d bytes", 
                 stack.calculate_total_overhead());
    end
endmodule
```

### Example 3: Employee Management System

```systemverilog
// Payroll system demonstrating polymorphism
class PayrollSystem;
    Employee employees[];
    
    function new();
        Manager mgr;
        Developer dev1, dev2;
        
        // Create different types of employees
        mgr = new("Alice Johnson", 1001, 75000.0, 0.20, 8);
        dev1 = new("Bob Smith", 1002, 65000.0, "SystemVerilog");
        dev2 = new("Carol Davis", 1003, 68000.0, "Python");
        
        // Add some completed projects for developers
        dev1.complete_project();
        dev1.complete_project();
        dev2.complete_project();
        dev2.complete_project();
        dev2.complete_project();
        
        // Store in polymorphic array
        employees = new[3];
        employees[0] = mgr;
        employees[1] = dev1;
        employees[2] = dev2;
    endfunction
    
    function void process_payroll();
        real total_payroll = 0.0;
        
        $display("=== Monthly Payroll Processing ===\n");
        
        foreach (employees[i]) begin
            real pay = employees[i].calculate_pay();  // Polymorphic call
            total_payroll += pay;
            
            employees[i].display_info();  // Polymorphic call
            $display("Monthly Pay: $%0.2f\n", pay);
        end
        
        $display("Total Monthly Payroll: $%0.2f", total_payroll);
    endfunction
    
    function void display_employee_details();
        $display("=== Employee Details ===\n");
        
        foreach (employees[i]) begin
            employees[i].display_info();
            
            // Type checking and casting for specific methods
            if ($cast(mgr_handle, employees[i])) begin
                Manager mgr_handle;
                mgr_handle.conduct_meeting();
            end else if ($cast(dev_handle, employees[i])) begin
                Developer dev_handle;
                dev_handle.complete_project();
            end
            $display("");
        end
    endfunction
endclass

// Test module for payroll system
module payroll_demo;
    PayrollSystem payroll;
    
    initial begin
        payroll = new();
        payroll.process_payroll();
        $display("\n" + {50{"="}});
        payroll.display_employee_details();
    end
endmodule
```

### **Example 16:** Pet Care System
**Description:** Array of Animal handles containing different pet types. Demonstrates polymorphic method calls for feeding and care routines.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_16__pet_care_system/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// pet_care_system.sv
package pet_care_pkg;

  // Enumeration for pet types
  typedef enum {DOG, CAT, FISH} pet_type_e;

  // Simple struct-based approach for Verilator compatibility
  typedef struct {
    string name;
    int age;
    pet_type_e pet_type;
    string breed_or_info;  // breed for dog, indoor/outdoor for cat, tank size for fish
  } pet_info_t;

  // Pet care functions
  function void feed_pet(pet_info_t pet);
    case(pet.pet_type)
      DOG:  $display("  Feeding %s (Dog) with dry kibble and fresh water", pet.name);
      CAT:  $display("  Feeding %s (Cat) with wet food and treats", pet.name);
      FISH: $display("  Feeding %s (Fish) with flakes and bloodworms", pet.name);
    endcase
  endfunction

  function void care_for_pet(pet_info_t pet);
    case(pet.pet_type)
      DOG:  $display("  Walking %s, playing fetch, and brushing fur", pet.name);
      CAT:  $display("  Cleaning %s's litter box, playing with toys (%s cat)", 
                     pet.name, pet.breed_or_info);
      FISH: $display("  Cleaning %s's %s tank and checking water pH", 
                     pet.name, pet.breed_or_info);
    endcase
  endfunction

  function string get_pet_info(pet_info_t pet);
    case(pet.pet_type)
      DOG:  return $sformatf("%s is a %0d-year-old %s", 
                            pet.name, pet.age, pet.breed_or_info);
      CAT:  return $sformatf("%s is a %0d-year-old %s cat", 
                            pet.name, pet.age, pet.breed_or_info);
      FISH: return $sformatf("%s is a %0d-year-old fish in a %s tank", 
                            pet.name, pet.age, pet.breed_or_info);
    endcase
  endfunction

endpackage

// Main design module
module pet_care_system;
  import pet_care_pkg::*;
  
  // Array of pet structures for polymorphic-like behavior
  pet_info_t pet_array[6];
  
  initial begin
    $display("=== Pet Care System Initialization ===");
    
    // Create different types of pets using struct initialization
    pet_array[0] = '{"Buddy", 5, DOG, "Golden Retriever"};
    pet_array[1] = '{"Whiskers", 3, CAT, "indoor"};
    pet_array[2] = '{"Goldie", 2, FISH, "10-gallon"};
    pet_array[3] = '{"Max", 7, DOG, "German Shepherd"};
    pet_array[4] = '{"Shadow", 4, CAT, "outdoor"};
    pet_array[5] = '{"Nemo", 1, FISH, "5-gallon"};
    
    // Display all pets information
    $display("\n=== Pet Information ===");
    for(int i = 0; i < 6; i++) begin
      $display("%0d. %s", i+1, get_pet_info(pet_array[i]));
    end
    
    // Demonstrate polymorphic-like feeding routine
    $display("\n=== Daily Feeding Routine ===");
    for(int i = 0; i < 6; i++) begin
      feed_pet(pet_array[i]);
    end
    
    // Demonstrate polymorphic-like care routine
    $display("\n=== Daily Care Routine ===");
    for(int i = 0; i < 6; i++) begin
      care_for_pet(pet_array[i]);
    end
    
    $display("\n=== Pet Care System Complete ===");
  end

endmodule
```

```systemverilog
// pet_care_system_testbench.sv
module pet_care_testbench;
  import pet_care_pkg::*;
  
  // Instantiate the design under test
  pet_care_system PET_CARE_INSTANCE();
  
  // Additional test pets
  pet_info_t test_pets[3];
  pet_info_t mixed_pets[4];
  
  initial begin
    // Setup VCD dumping for waveform analysis
    $dumpfile("pet_care_testbench.vcd");
    $dumpvars(0, pet_care_testbench);
    
    #10;  // Wait for design to initialize
    
    $display("\n=== Testbench: Additional Pet Care Scenarios ===");
    
    // Test scenario 1: Creating pets with different parameters
    test_pets[0] = '{"Rex", 2, DOG, "Labrador"};
    test_pets[1] = '{"Mittens", 6, CAT, "indoor"};
    test_pets[2] = '{"Bubbles", 3, FISH, "20-gallon"};
    
    $display("\n--- Test Scenario 1: New Pet Registration ---");
    for(int i = 0; i < 3; i++) begin
      $display("Registering: %s", get_pet_info(test_pets[i]));
    end
    
    // Test scenario 2: Special care routine for older pets
    $display("\n--- Test Scenario 2: Senior Pet Care ---");
    for(int i = 0; i < 3; i++) begin
      if(test_pets[i].age >= 5) begin
        $display("Senior pet detected: %s", get_pet_info(test_pets[i]));
        $display("  Providing extra care and health monitoring");
        care_for_pet(test_pets[i]);
      end
    end
    
    // Test scenario 3: Feeding schedule verification
    $display("\n--- Test Scenario 3: Feeding Schedule ---");
    $display("Morning feeding routine:");
    for(int i = 0; i < 3; i++) begin
      feed_pet(test_pets[i]);
    end
    
    #5;  // Small delay
    
    // Test scenario 4: Pet type verification
    $display("\n--- Test Scenario 4: Pet Type Analysis ---");
    for(int i = 0; i < 3; i++) begin
      case(test_pets[i].pet_type)
        DOG:  $display("%s is a canine companion", test_pets[i].name);
        CAT:  $display("%s is a feline friend", test_pets[i].name);
        FISH: $display("%s is an aquatic pet", test_pets[i].name);
      endcase
    end
    
    // Test scenario 5: Demonstrate function-based polymorphism
    $display("\n--- Test Scenario 5: Function-Based Care System ---");
    mixed_pets[0] = '{"Spot", 3, DOG, "Beagle"};
    mixed_pets[1] = '{"Fluffy", 2, CAT, "indoor"};
    mixed_pets[2] = '{"Splash", 1, FISH, "15-gallon"};
    mixed_pets[3] = '{"Tiger", 8, CAT, "outdoor"};
    
    $display("Processing mixed pet group:");
    for(int i = 0; i < 4; i++) begin
      $display("Pet %0d: %s", i+1, get_pet_info(mixed_pets[i]));
      feed_pet(mixed_pets[i]);
      care_for_pet(mixed_pets[i]);
      $display("");
    end
    
    #10;  // Final delay
    
    $display("=== Testbench: Pet Care System Verification Complete ===");
    $display("All polymorphic-like function calls executed successfully!");
    
  end

endmodule
```

Verilator Simulation Output:
=== Pet Care System Initialization ===

=== Pet Information ===
1. Buddy is a 5-year-old Golden Retriever
2. Whiskers is a 3-year-old indoor cat
3. Goldie is a 2-year-old fish in a 10-gallon tank
4. Max is a 7-year-old German Shepherd
5. Shadow is a 4-year-old outdoor cat
6. Nemo is a 1-year-old fish in a 5-gallon tank

=== Daily Feeding Routine ===
  Feeding Buddy (Dog) with dry kibble and fresh water
  Feeding Whiskers (Cat) with wet food and treats
  Feeding Goldie (Fish) with flakes and bloodworms
  Feeding Max (Dog) with dry kibble and fresh water
  Feeding Shadow (Cat) with wet food and treats
  Feeding Nemo (Fish) with flakes and bloodworms

=== Daily Care Routine ===
  Walking Buddy, playing fetch, and brushing fur
  Cleaning Whiskers's litter box, playing with toys (indoor cat)
  Cleaning Goldie's 10-gallon tank and checking water pH
  Walking Max, playing fetch, and brushing fur
  Cleaning Shadow's litter box, playing with toys (outdoor cat)
  Cle

0

### **Example 17:** Simple Media Player
**Description:** MediaFile array with different file types (Audio, Video, Image). Shows polymorphic play() method calls for different media types.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_17__simple_media_player/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// media_player_system.sv
package media_player_pkg;

  // Base class for all media files
  virtual class MediaFile;
    protected string filename;
    protected int file_size;
    protected string format;
    
    function new(string fname, int fsize, string fmt);
      this.filename = fname;
      this.file_size = fsize;
      this.format = fmt;
    endfunction
    
    // Pure virtual function - must be implemented by derived classes
    pure virtual function void play();
    
    // Virtual function that can be overridden
    virtual function void display_info();
      $display("File: %s, Size: %0d KB, Format: %s", 
               filename, file_size, format);
    endfunction
    
    function string get_filename();
      return filename;
    endfunction
    
    function int get_file_size();
      return file_size;
    endfunction
    
  endclass

  // Audio file class
  class AudioFile extends MediaFile;
    protected int bitrate;
    protected int duration; // in seconds
    
    function new(string fname, int fsize, string fmt, int br, int dur);
      super.new(fname, fsize, fmt);
      this.bitrate = br;
      this.duration = dur;
    endfunction
    
    virtual function void play();
      $display("Playing audio: %s", filename);
      $display("   Bitrate: %0d kbps, Duration: %0d:%02d", 
               bitrate, duration/60, duration%60);
      $display("Audio streaming started...");
    endfunction
    
    virtual function void display_info();
      super.display_info();
      $display("   Audio Info - Bitrate: %0d kbps, Duration: %0d:%02d",
               bitrate, duration/60, duration%60);
    endfunction
    
  endclass

  // Video file class
  class VideoFile extends MediaFile;
    protected string resolution;
    protected int frame_rate;
    protected int duration; // in seconds
    
    function new(string fname, int fsize, string fmt, string res, 
                 int fps, int dur);
      super.new(fname, fsize, fmt);
      this.resolution = res;
      this.frame_rate = fps;
      this.duration = dur;
    endfunction
    
    virtual function void play();
      $display("Playing video: %s", filename);
      $display("   Resolution: %s, Frame Rate: %0d fps, Duration: %0d:%02d",
               resolution, frame_rate, duration/60, duration%60);
      $display("Video playback started...");
    endfunction
    
    virtual function void display_info();
      super.display_info();
      $display("   Video Info - Resolution: %s, FPS: %0d, Duration: %0d:%02d",
               resolution, frame_rate, duration/60, duration%60);
    endfunction
    
  endclass

  // Image file class
  class ImageFile extends MediaFile;
    protected string resolution;
    protected int color_depth;
    
    function new(string fname, int fsize, string fmt, string res, int depth);
      super.new(fname, fsize, fmt);
      this.resolution = res;
      this.color_depth = depth;
    endfunction
    
    virtual function void play();
      $display("Displaying image: %s", filename);
      $display("   Resolution: %s, Color Depth: %0d bits", 
               resolution, color_depth);
      $display("Image displayed on screen...");
    endfunction
    
    virtual function void display_info();
      super.display_info();
      $display("   Image Info - Resolution: %s, Color Depth: %0d bits",
               resolution, color_depth);
    endfunction
    
  endclass

  // Media player class that manages a collection of media files
  class MediaPlayer;
    protected MediaFile media_library[$];
    protected string player_name;
    
    function new(string name);
      this.player_name = name;
    endfunction
    
    function void add_media_file(MediaFile media);
      media_library.push_back(media);
      $display("Added '%s' to %s media library", 
               media.get_filename(), player_name);
    endfunction
    
    function void play_all_media();
      $display("\n=== %s: Playing All Media Files ===", player_name);
      foreach(media_library[i]) begin
        $display("\n[%0d] ", i+1);
        media_library[i].play();
      end
    endfunction
    
    function void display_library();
      $display("\n=== %s: Media Library ===", player_name);
      if(media_library.size() == 0) begin
        $display("Library is empty.");
        return;
      end
      
      foreach(media_library[i]) begin
        $display("\n[%0d] ", i+1);
        media_library[i].display_info();
      end
    endfunction
    
    function void play_media_by_index(int index);
      if(index >= 0 && index < media_library.size()) begin
        $display("\n=== Playing Media File %0d ===", index+1);
        media_library[index].play();
      end else begin
        $display("Invalid media index: %0d", index+1);
      end
    endfunction
    
    function int get_library_size();
      return media_library.size();
    endfunction
    
  endclass

endpackage

module media_player_system;
  import media_player_pkg::*;
  
  initial begin
    $display("=== Simple Media Player System ===");
    $display("Demonstrating polymorphism with different media types");
    $display();
  end
  
endmodule
```

```systemverilog
// media_player_system_testbench.sv
module media_player_testbench;
  import media_player_pkg::*;
  
  // Instantiate the design under test
  media_player_system MEDIA_PLAYER_INSTANCE();
  
  initial begin
    // Dump waves for debugging
    $dumpfile("media_player_testbench.vcd");
    $dumpvars(0, media_player_testbench);
    
    // Test the media player functionality
    test_media_player();
    
    #10; // Wait a bit before finishing
    $display("\nTestbench completed successfully!");
    $finish;
  end
  
  // Task to test media player functionality
  task test_media_player();
    // Create media player instance
    MediaPlayer my_player;
    
    // Create different types of media files
    AudioFile song1, song2;
    VideoFile movie1, movie2;
    ImageFile photo1, photo2;
    
    // Array to demonstrate polymorphism
    MediaFile mixed_media[$];
    
    $display("=== Creating Media Player ===");
    my_player = new("MyAwesome Player");
    
    $display("\n=== Creating Media Files ===");
    // Create audio files
    song1 = new("favorite_song.mp3", 4500, "MP3", 320, 215);
    song2 = new("classical_piece.flac", 35000, "FLAC", 1411, 180);
    
    // Create video files  
    movie1 = new("action_movie.mp4", 2100000, "MP4", "1920x1080", 24, 7200);
    movie2 = new("documentary.mkv", 1800000, "MKV", "1280x720", 30, 5400);
    
    // Create image files
    photo1 = new("vacation_photo.jpg", 2500, "JPEG", "4032x3024", 24);
    photo2 = new("wallpaper.png", 8000, "PNG", "3840x2160", 32);
    
    $display("\n=== Adding Media to Player ===");
    // Add media files to player
    my_player.add_media_file(song1);
    my_player.add_media_file(movie1);
    my_player.add_media_file(photo1);
    my_player.add_media_file(song2);
    my_player.add_media_file(movie2);
    my_player.add_media_file(photo2);
    
    // Display library contents
    my_player.display_library();
    
    $display("\n=== Testing Individual Media Playback ===");
    // Play specific media files
    my_player.play_media_by_index(0); // Audio
    my_player.play_media_by_index(1); // Video
    my_player.play_media_by_index(2); // Image
    
    // Play all media files (demonstrates polymorphism)
    my_player.play_all_media();
    
    $display("\n=== Demonstrating Polymorphism with Mixed Array ===");
    // Create mixed array of different media types
    mixed_media.push_back(song1);
    mixed_media.push_back(movie1);
    mixed_media.push_back(photo1);
    mixed_media.push_back(song2);
    
    $display("Playing mixed media types using polymorphic calls:");
    foreach(mixed_media[i]) begin
      $display("\n--- Mixed Media Item %0d ---", i+1);
      mixed_media[i].display_info();
      mixed_media[i].play();
    end
    
    $display("\n=== Testing Error Handling ===");
    // Test invalid index
    my_player.play_media_by_index(99);
    
    $display("\n=== Media Player Demo Complete ===");
    $display("Library contains %0d media files", my_player.get_library_size());
    $display("Demonstrated polymorphic behavior with different media types");
    
  endtask
  
endmodule
```

Verilator Simulation Output:
=== Simple Media Player System ===
Demonstrating polymorphism with different media types

=== Creating Media Player ===

=== Creating Media Files ===

=== Adding Media to Player ===
Added 'favorite_song.mp3' to MyAwesome Player media library
Added 'action_movie.mp4' to MyAwesome Player media library
Added 'vacation_photo.jpg' to MyAwesome Player media library
Added 'classical_piece.flac' to MyAwesome Player media library
Added 'documentary.mkv' to MyAwesome Player media library
Added 'wallpaper.png' to MyAwesome Player media library

=== MyAwesome Player: Media Library ===

[1]
File: favorite_song.mp3, Size: 4500 KB, Format: MP3
   Audio Info - Bitrate: 320 kbps, Duration: 3:35

[2]
File: action_movie.mp4, Size: 2100000 KB, Format: MP4
   Video Info - Resolution: 1920x1080, FPS: 24, Duration: 120:00

[3]
File: vacation_photo.jpg, Size: 2500 KB, Format: JPEG
   Image Info - Resolution: 4032x3024, Color Depth: 24 bits

[4]
File: classical_piece.flac, Size: 35

0

### **Example 18:** Basic Shape Calculator
**Description:** Collection of Shape objects calculating total area. Demonstrates polymorphic area calculation for mixed shape types.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_18__shape_calculator/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// shape_calculator.sv
package shape_pkg;

  // Base abstract shape class
  virtual class Shape;
    pure virtual function real calculate_area();
    pure virtual function string get_name();
  endclass

  // Circle class
  class Circle extends Shape;
    real radius;
    
    function new(real r);
      radius = r;
    endfunction
    
    virtual function real calculate_area();
      return 3.14159 * radius * radius;
    endfunction
    
    virtual function string get_name();
      return "Circle";
    endfunction
  endclass

  // Rectangle class
  class Rectangle extends Shape;
    real width, height;
    
    function new(real w, real h);
      width = w;
      height = h;
    endfunction
    
    virtual function real calculate_area();
      return width * height;
    endfunction
    
    virtual function string get_name();
      return "Rectangle";
    endfunction
  endclass

  // Triangle class
  class Triangle extends Shape;
    real base, height;
    
    function new(real b, real h);
      base = b;
      height = h;
    endfunction
    
    virtual function real calculate_area();
      return 0.5 * base * height;
    endfunction
    
    virtual function string get_name();
      return "Triangle";
    endfunction
  endclass

  // Shape calculator class
  class ShapeCalculator;
    Shape shape_collection[];
    int shape_count;
    
    function new();
      shape_count = 0;
    endfunction
    
    function void add_shape(Shape s);
      shape_collection = new[shape_count + 1](shape_collection);
      shape_collection[shape_count] = s;
      shape_count++;
    endfunction
    
    function real calculate_total_area();
      real total = 0.0;
      for (int i = 0; i < shape_count; i++) begin
        total += shape_collection[i].calculate_area();
      end
      return total;
    endfunction
    
    function void display_shapes();
      $display("Shape Collection:");
      $display("================");
      for (int i = 0; i < shape_count; i++) begin
        $display("%0d. %s - Area: %.2f", i+1, 
                 shape_collection[i].get_name(),
                 shape_collection[i].calculate_area());
      end
      $display("================");
      $display("Total Area: %.2f", calculate_total_area());
    endfunction
  endclass

endpackage

module shape_calculator_design;
  import shape_pkg::*;
  
  initial begin
    ShapeCalculator calc;
    Circle circle1, circle2;
    Rectangle rect1, rect2;
    Triangle triangle1;
    
    $display();
    $display("Basic Shape Calculator Example");
    $display("==============================");
    
    // Create calculator instance
    calc = new();
    
    // Create various shapes
    circle1 = new(5.0);    // radius = 5.0
    circle2 = new(3.0);    // radius = 3.0
    rect1 = new(4.0, 6.0); // width = 4.0, height = 6.0
    rect2 = new(2.5, 8.0); // width = 2.5, height = 8.0
    triangle1 = new(6.0, 4.0);  // base = 6.0, height = 4.0
    
    // Add shapes to calculator
    calc.add_shape(circle1);
    calc.add_shape(rect1);
    calc.add_shape(triangle1);
    calc.add_shape(circle2);
    calc.add_shape(rect2);
    
    // Display all shapes and total area
    calc.display_shapes();
    
    $display();
    $display("Polymorphic area calculation completed!");
    $display();
  end
endmodule
```

```systemverilog
// shape_calculator_testbench.sv
module shape_calculator_testbench;
  import shape_pkg::*;
  
  // Instantiate design under test
  shape_calculator_design DESIGN_INSTANCE();
  
  // Additional testbench verification
  initial begin
    ShapeCalculator test_calc;
    Circle test_circle;
    Rectangle test_rect;
    Triangle test_triangle;
    real expected_area, actual_area;
    
    // Dump waves
    $dumpfile("shape_calculator_testbench.vcd");
    $dumpvars(0, shape_calculator_testbench);
    
    #1; // Wait for design to complete
    
    $display("Hello from testbench!");
    $display("Running verification tests...");
    $display();
    
    // Test individual shape calculations
    test_circle = new(2.0);
    expected_area = 3.14159 * 2.0 * 2.0; // Ï€ * rÂ²
    actual_area = test_circle.calculate_area();
    $display("Circle Test: Expected=%.2f, Actual=%.2f %s", 
             expected_area, actual_area, 
             (actual_area == expected_area) ? "PASS" : "FAIL");
    
    test_rect = new(3.0, 4.0);
    expected_area = 3.0 * 4.0; // width * height
    actual_area = test_rect.calculate_area();
    $display("Rectangle Test: Expected=%.2f, Actual=%.2f %s", 
             expected_area, actual_area, 
             (actual_area == expected_area) ? "PASS" : "FAIL");
    
    test_triangle = new(6.0, 2.0);
    expected_area = 0.5 * 6.0 * 2.0; // 0.5 * base * height
    actual_area = test_triangle.calculate_area();
    $display("Triangle Test: Expected=%.2f, Actual=%.2f %s", 
             expected_area, actual_area, 
             (actual_area == expected_area) ? "PASS" : "FAIL");
    
    $display();
    
    // Test mixed shape collection
    test_calc = new();
    test_calc.add_shape(test_circle);
    test_calc.add_shape(test_rect);
    test_calc.add_shape(test_triangle);
    
    expected_area = (3.14159 * 4.0) + 12.0 + 6.0; // Sum of all areas
    actual_area = test_calc.calculate_total_area();
    $display("Total Area Test: Expected=%.2f, Actual=%.2f %s", 
             expected_area, actual_area, 
             (actual_area == expected_area) ? "PASS" : "FAIL");
    
    $display();
    $display("Verification complete!");
    $display();
    
    #10;
    $finish;
  end
endmodule
```

Verilator Simulation Output:

Basic Shape Calculator Example
Shape Collection:
1. Circle - Area: 78.54
2. Rectangle - Area: 24.00
3. Triangle - Area: 12.00
4. Circle - Area: 28.27
5. Rectangle - Area: 20.00
Total Area: 162.81

Polymorphic area calculation completed!

Hello from testbench!
Running verification tests...

Circle Test: Expected=12.57, Actual=12.57 PASS
Rectangle Test: Expected=12.00, Actual=12.00 PASS
Triangle Test: Expected=6.00, Actual=6.00 PASS

Total Area Test: Expected=30.57, Actual=30.57 PASS

Verification complete!

Process finished with return code: 0
Removing Chapter_10_examples/example_18__shape_calculator/obj_dir directory...
Chapter_10_examples/example_18__shape_calculator/obj_dir removed successfully.


0

### **Example 19:** Simple Command Processor
**Description:** Command base class with execute() method. Different command types (Save, Load, Delete) stored in array and executed polymorphically.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_19__simple_command_processor/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


Verilator Simulation Output:
=== Command Processor Design Module ===

--- Adding Commands ---
[0] Added SAVE command to queue (total: 1)
[0] Added LOAD command to queue (total: 2)
[0] Added DELETE command to queue (total: 3)
[0] Added SAVE command to queue (total: 4)
[0] Added LOAD command to queue (total: 5)

--- Executing Commands ---
[0] Starting command execution (5 commands)
[0] Command 1 of 5:
[0] Executing SAVE command: Saving to file 'config.txt'
[0] Command 2 of 5:
[0] Executing LOAD command: Loading from file 'settings.cfg'
[0] Command 3 of 5:
[0] Executing DELETE command: Deleting file 'temp.log'
[0] Command 4 of 5:
[0] Executing SAVE command: Saving to file 'backup.dat'
[0] Command 5 of 5:
[0] Executing LOAD command: Loading from file 'profile.xml'
[0] All commands executed successfully

--- Final Status ---
Queue size: 5
[0] Command queue cleared
Queue size after clear: 0

=== Design Module Complete ===
=== Command Processor Testbench ===

--- Testbench Additional Tests --

0

### **Example 20:** Basic Vehicle Fleet
**Description:** Vehicle array with cars, trucks, and motorcycles. Shows polymorphic fuel calculation and maintenance scheduling.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_20__vehicle_fleet/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// vehicle_fleet_system.sv
package vehicle_fleet_pkg;

  // Base Vehicle class with virtual methods for polymorphism
  virtual class Vehicle;
    string vehicle_id;
    real mileage;
    real fuel_capacity;
    real fuel_level;
    real last_maintenance_miles;
    
    function new(string id, real capacity);
      vehicle_id = id;
      fuel_capacity = capacity;
      fuel_level = capacity;
      mileage = 0.0;
      last_maintenance_miles = 0.0;
    endfunction
    
    // Pure virtual methods - must be implemented by derived classes
    pure virtual function real calculate_fuel_consumption(real distance);
    pure virtual function int get_maintenance_interval();
    pure virtual function string get_vehicle_type();
    
    // Common methods for all vehicles
    function void drive(real distance);
      real fuel_consumed = calculate_fuel_consumption(distance);
      mileage += distance;
      fuel_level -= fuel_consumed;
      $display("  %s drove %.1f miles, fuel level: %.1f/%.1f", 
               vehicle_id, distance, fuel_level, fuel_capacity);
    endfunction
    
    function bit needs_maintenance();
      real interval = get_maintenance_interval();
      return (mileage - last_maintenance_miles) >= interval;
    endfunction
    
    function void perform_maintenance();
      last_maintenance_miles = mileage;
      $display("  %s maintenance completed at %.1f miles", 
               vehicle_id, mileage);
    endfunction
    
    function void refuel();
      fuel_level = fuel_capacity;
      $display("  %s refueled to %.1f gallons", vehicle_id, fuel_capacity);
    endfunction
    
    function void display_status();
      $display("  %s (%s): %.1f miles, fuel: %.1f/%.1f, maintenance: %s", 
               vehicle_id, get_vehicle_type(), mileage, fuel_level, 
               fuel_capacity, needs_maintenance() ? "NEEDED" : "OK");
    endfunction
  endclass

  // Car class - good fuel efficiency, frequent maintenance
  class Car extends Vehicle;
    function new(string id);
      super.new(id, 15.0); // 15 gallon tank
    endfunction
    
    function real calculate_fuel_consumption(real distance);
      return distance / 30.0; // 30 MPG
    endfunction
    
    function int get_maintenance_interval();
      return 5000; // Every 5000 miles
    endfunction
    
    function string get_vehicle_type();
      return "Car";
    endfunction
  endclass

  // Truck class - poor fuel efficiency, less frequent maintenance
  class Truck extends Vehicle;
    function new(string id);
      super.new(id, 25.0); // 25 gallon tank
    endfunction
    
    function real calculate_fuel_consumption(real distance);
      return distance / 15.0; // 15 MPG
    endfunction
    
    function int get_maintenance_interval();
      return 8000; // Every 8000 miles
    endfunction
    
    function string get_vehicle_type();
      return "Truck";
    endfunction
  endclass

  // Motorcycle class - excellent fuel efficiency, frequent maintenance
  class Motorcycle extends Vehicle;
    function new(string id);
      super.new(id, 4.0); // 4 gallon tank
    endfunction
    
    function real calculate_fuel_consumption(real distance);
      return distance / 45.0; // 45 MPG
    endfunction
    
    function int get_maintenance_interval();
      return 3000; // Every 3000 miles
    endfunction
    
    function string get_vehicle_type();
      return "Motorcycle";
    endfunction
  endclass

  // Fleet management class
  class VehicleFleet;
    Vehicle fleet[$]; // Dynamic array of vehicles
    
    function void add_vehicle(Vehicle v);
      fleet.push_back(v);
      $display("Added %s (%s) to fleet", v.vehicle_id, v.get_vehicle_type());
    endfunction
    
    function void drive_all_vehicles(real distance);
      $display("\n--- Driving all vehicles %.1f miles ---", distance);
      foreach (fleet[i]) begin
        fleet[i].drive(distance);
      end
    endfunction
    
    function void check_maintenance_needs();
      $display("\n--- Checking maintenance needs ---");
      foreach (fleet[i]) begin
        if (fleet[i].needs_maintenance()) begin
          $display("  %s needs maintenance!", fleet[i].vehicle_id);
          fleet[i].perform_maintenance();
        end else begin
          $display("  %s maintenance OK", fleet[i].vehicle_id);
        end
      end
    endfunction
    
    function void refuel_all_vehicles();
      $display("\n--- Refueling all vehicles ---");
      foreach (fleet[i]) begin
        fleet[i].refuel();
      end
    endfunction
    
    function void display_fleet_status();
      $display("\n--- Fleet Status ---");
      foreach (fleet[i]) begin
        fleet[i].display_status();
      end
    endfunction
    
    function real calculate_total_fuel_consumption(real distance);
      real total_fuel = 0.0;
      foreach (fleet[i]) begin
        total_fuel += fleet[i].calculate_fuel_consumption(distance);
      end
      return total_fuel;
    endfunction
  endclass

endpackage

module vehicle_fleet_system;
  import vehicle_fleet_pkg::*;
  
  // Fleet management system demonstration
  initial begin
    VehicleFleet fleet;
    Car car1, car2;
    Truck truck1;
    Motorcycle bike1;
    real trip_distance = 100.0;
    
    $display("=== Vehicle Fleet Management System ===");
    
    // Create fleet and vehicles
    fleet = new();
    car1 = new("CAR-001");
    car2 = new("CAR-002");
    truck1 = new("TRUCK-001");
    bike1 = new("BIKE-001");
    
    // Add vehicles to fleet
    $display("\n--- Building Fleet ---");
    fleet.add_vehicle(car1);
    fleet.add_vehicle(car2);
    fleet.add_vehicle(truck1);
    fleet.add_vehicle(bike1);
    
    // Show initial status
    fleet.display_fleet_status();
    
    // Calculate fuel consumption for a trip
    $display("\n--- Fuel Consumption Analysis ---");
    $display("Total fuel needed for %.1f mile trip: %.2f gallons", 
             trip_distance, fleet.calculate_total_fuel_consumption(trip_distance));
    
    // Simulate driving
    fleet.drive_all_vehicles(trip_distance);
    fleet.display_fleet_status();
    
    // Drive more to trigger maintenance
    fleet.drive_all_vehicles(2500.0);
    fleet.drive_all_vehicles(2500.0);
    fleet.check_maintenance_needs();
    
    // Refuel all vehicles
    fleet.refuel_all_vehicles();
    fleet.display_fleet_status();
    
    $display("\n=== Fleet Management Demo Complete ===");
  end

endmodule
```

```systemverilog
// vehicle_fleet_system_testbench.sv
module vehicle_fleet_testbench;
  import vehicle_fleet_pkg::*;
  
  // Instantiate the design under test
  vehicle_fleet_system fleet_system_instance();
  
  // Additional test scenarios
  initial begin
    VehicleFleet test_fleet;
    Car economy_car;
    Truck heavy_truck;
    Motorcycle sport_bike;
    
    // Setup wave dumping
    $dumpfile("vehicle_fleet_testbench.vcd");
    $dumpvars(0, vehicle_fleet_testbench);
    
    // Wait for main system to complete
    #1;
    
    $display("\n\n=== Additional Test Scenarios ===");
    
    // Test polymorphic behavior with different vehicle types
    test_fleet = new();
    economy_car = new("ECON-001");
    heavy_truck = new("HEAVY-001");
    sport_bike = new("SPORT-001");
    
    test_fleet.add_vehicle(economy_car);
    test_fleet.add_vehicle(heavy_truck);
    test_fleet.add_vehicle(sport_bike);
    
    // Test fuel efficiency comparison
    $display("\n--- Fuel Efficiency Comparison (200 miles) ---");
    $display("Car fuel consumption: %.2f gallons", 
             economy_car.calculate_fuel_consumption(200.0));
    $display("Truck fuel consumption: %.2f gallons", 
             heavy_truck.calculate_fuel_consumption(200.0));
    $display("Motorcycle fuel consumption: %.2f gallons", 
             sport_bike.calculate_fuel_consumption(200.0));
    
    // Test maintenance intervals
    $display("\n--- Maintenance Interval Comparison ---");
    $display("Car maintenance interval: %0d miles", 
             economy_car.get_maintenance_interval());
    $display("Truck maintenance interval: %0d miles", 
             heavy_truck.get_maintenance_interval());
    $display("Motorcycle maintenance interval: %0d miles", 
             sport_bike.get_maintenance_interval());
    
    // Test long-distance driving scenario
    $display("\n--- Long Distance Driving Test ---");
    test_fleet.drive_all_vehicles(3000.0);
    test_fleet.check_maintenance_needs();
    
    // Test multiple maintenance cycles
    $display("\n--- Multiple Maintenance Cycles ---");
    test_fleet.drive_all_vehicles(5000.0);
    test_fleet.check_maintenance_needs();
    test_fleet.drive_all_vehicles(3000.0);
    test_fleet.check_maintenance_needs();
    
    // Final fleet status
    test_fleet.display_fleet_status();
    
    $display("\n=== Testbench Complete ===");
    
    // Small delay before finish
    #10;
    $finish;
  end
  
  // Monitor for any potential issues
  initial begin
    #1000; // Timeout after 1000 time units
    $display("WARNING: Testbench timeout reached");
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Vehicle Fleet Management System ===

--- Building Fleet ---
Added CAR-001 (Car) to fleet
Added CAR-002 (Car) to fleet
Added TRUCK-001 (Truck) to fleet
Added BIKE-001 (Motorcycle) to fleet

--- Fleet Status ---
  CAR-001 (Car): 0.0 miles, fuel: 15.0/15.0, maintenance:     OK
  CAR-002 (Car): 0.0 miles, fuel: 15.0/15.0, maintenance:     OK
  TRUCK-001 (Truck): 0.0 miles, fuel: 25.0/25.0, maintenance:     OK
  BIKE-001 (Motorcycle): 0.0 miles, fuel: 4.0/4.0, maintenance:     OK

--- Fuel Consumption Analysis ---
Total fuel needed for 100.0 mile trip: 15.56 gallons

--- Driving all vehicles 100.0 miles ---
  CAR-001 drove 100.0 miles, fuel level: 11.7/15.0
  CAR-002 drove 100.0 miles, fuel level: 11.7/15.0
  TRUCK-001 drove 100.0 miles, fuel level: 18.3/25.0
  BIKE-001 drove 100.0 miles, fuel level: 1.8/4.0

--- Fleet Status ---
  CAR-001 (Car): 100.0 miles, fuel: 11.7/15.0, maintenance:     OK
  CAR-002 (Car): 100.0 miles, fuel: 11.7/15.0, maintenance:    

0

## Best Practices and Design Patterns

### Liskov Substitution Principle
Objects of derived classes should be substitutable for objects of the base class without altering program correctness.

```systemverilog
// Good: Circle can substitute Shape
Shape my_shape = new Circle(5.0);
real area = my_shape.calculate_area();  // Works correctly
```

### Interface Segregation
Keep interfaces focused and cohesive.

```systemverilog
// Instead of one large interface, use specific interfaces
class Drawable;
    pure virtual function void draw();
endclass

class Calculable;
    pure virtual function real calculate_area();
endclass

class Circle extends Drawable, Calculable;  // Multiple inheritance
    // Implementation
endclass
```

### Factory Pattern Example

```systemverilog
class ShapeFactory;
    static function Shape create_shape(string shape_type, real param1 = 1.0, real param2 = 1.0);
        case (shape_type.tolower())
            "circle": return new Circle(param1);
            "rectangle": return new Rectangle(param1, param2);
            "triangle": return new Triangle(param1, param2);
            default: begin
                $display("Unknown shape type: %s", shape_type);
                return null;
            end
        endcase
    endfunction
endclass

// Usage
Shape my_shape = ShapeFactory::create_shape("circle", 3.0);
```

### **Example 21:** Simple Factory Pattern
**Description:** AnimalFactory creates different animals based on string input. Demonstrates object creation without exposing instantiation logic.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_21__simple_factory_pattern/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// animal_factory_design.sv
package animal_factory_pkg;

  // Simple factory pattern using basic types
  typedef enum {ANIMAL_DOG, ANIMAL_CAT, ANIMAL_BIRD, ANIMAL_UNKNOWN} animal_type_e;
  
  // Factory function - creates animal type from string
  function animal_type_e create_animal(string animal_type_str);
    case (animal_type_str.tolower())
      "dog":  return ANIMAL_DOG;
      "cat":  return ANIMAL_CAT;
      "bird": return ANIMAL_BIRD;
      default: begin
        $display("Unknown animal type: %s", animal_type_str);
        return ANIMAL_UNKNOWN;
      end
    endcase
  endfunction
  
  // Get animal name from type
  function string get_animal_name(animal_type_e animal_type);
    case (animal_type)
      ANIMAL_DOG:  return "Dog";
      ANIMAL_CAT:  return "Cat";
      ANIMAL_BIRD: return "Bird";
      default:     return "Unknown";
    endcase
  endfunction
  
  // Get animal sound from type
  function string get_animal_sound(animal_type_e animal_type);
    case (animal_type)
      ANIMAL_DOG:  return "Woof!";
      ANIMAL_CAT:  return "Meow!";
      ANIMAL_BIRD: return "Tweet!";
      default:     return "Silence...";
    endcase
  endfunction
  
  // Check if animal is valid
  function bit is_valid_animal(animal_type_e animal_type);
    return (animal_type != ANIMAL_UNKNOWN);
  endfunction

endpackage

module animal_factory_design;
  import animal_factory_pkg::*;
  
  initial begin
    $display("Animal Factory Design Module Loaded");
  end
endmodule
```

```systemverilog
// animal_factory_design_testbench.sv
module animal_factory_testbench;
  import animal_factory_pkg::*;
  
  // Instantiate design under test
  animal_factory_design DESIGN_INSTANCE_NAME();
  
  initial begin
    // Dump waves
    $dumpfile("animal_factory_testbench.vcd");
    $dumpvars(0, animal_factory_testbench);
    
    $display();
    $display("=== Simple Factory Pattern Demonstration ===");
    $display();
    
    // Test creating different animals using the factory
    test_animal_creation("dog");
    test_animal_creation("cat");
    test_animal_creation("bird");
    test_animal_creation("fish");
    
    // Demonstrate case insensitive creation
    $display("=== Case Insensitive Test ===");
    test_animal_creation("DOG");
    
    $display();
    $display("Factory pattern demonstration complete!");
    
    #1;  // Wait for a time unit
    $finish;
  end
  
  // Helper task to test animal creation
  task test_animal_creation(string animal_str);
    animal_type_e my_animal;
    
    $display("Creating animal: %s", animal_str);
    my_animal = create_animal(animal_str);
    
    if (is_valid_animal(my_animal)) begin
      $display("  Animal created: %s", get_animal_name(my_animal));
      $display("  Sound: %s", get_animal_sound(my_animal));
    end else begin
      $display("  Failed to create animal!");
    end
    $display();
  endtask

endmodule
```

Verilator Simulation Output:
Animal Factory Design Module Loaded

=== Simple Factory Pattern Demonstration ===

Creating animal: dog
  Animal created: Dog
  Sound: Woof!

Creating animal: cat
  Animal created: Cat
  Sound: Meow!

Creating animal: bird
  Animal created: Bird
  Sound: Tweet!

Creating animal: fish
Unknown animal type: fish
  Failed to create animal!

=== Case Insensitive Test ===
Creating animal: DOG
  Animal created: Dog
  Sound: Woof!


Factory pattern demonstration complete!
Process finished with return code: 0
Removing Chapter_10_examples/example_21__simple_factory_pattern/obj_dir directory...
Chapter_10_examples/example_21__simple_factory_pattern/obj_dir removed successfully.


0

### **Example 22:** Basic Observer Pattern
**Description:** Simple WeatherStation notifies multiple Display objects. Shows basic publisher-subscriber relationship using inheritance.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_22__basic_observer_pattern/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// weather_station_observer.sv
package weather_observer_pkg;
  
  // Observer interface - defines the contract for observers
  virtual class observer_interface;
    pure virtual function void update(real temperature, real humidity, 
                                     real pressure);
  endclass
  
  // Subject interface - defines the contract for subjects
  virtual class subject_interface;
    protected observer_interface observers[$];
    
    virtual function void attach_observer(observer_interface obs);
      observers.push_back(obs);
      $display("[SUBJECT] Observer attached. Total observers: %0d", 
               observers.size());
    endfunction
    
    virtual function void detach_observer(observer_interface obs);
      for (int i = 0; i < observers.size(); i++) begin
        if (observers[i] == obs) begin
          observers.delete(i);
          $display("[SUBJECT] Observer detached. Remaining: %0d", 
                   observers.size());
          break;
        end
      end
    endfunction
    
    virtual function void notify_observers();
      // To be implemented by concrete subjects
    endfunction
  endclass
  
  // Concrete Subject - WeatherStation
  class weather_station extends subject_interface;
    protected real temperature;
    protected real humidity;
    protected real pressure;
    
    function new();
      temperature = 0.0;
      humidity = 0.0;
      pressure = 0.0;
    endfunction
    
    function void set_measurements(real temp, real humid, real press);
      temperature = temp;
      humidity = humid;
      pressure = press;
      $display("[WEATHER_STATION] New measurements recorded:");
      $display("  Temperature: %0.1f C", temperature);
      $display("  Humidity: %0.1f%%", humidity);
      $display("  Pressure: %0.1f hPa", pressure);
      notify_observers();
    endfunction
    
    virtual function void notify_observers();
      $display("[WEATHER_STATION] Notifying %0d observers...", 
               observers.size());
      foreach (observers[i]) begin
        observers[i].update(temperature, humidity, pressure);
      end
    endfunction
    
    function real get_temperature();
      return temperature;
    endfunction
    
    function real get_humidity();
      return humidity;
    endfunction
    
    function real get_pressure();
      return pressure;
    endfunction
  endclass
  
  // Concrete Observer - Current Conditions Display
  class current_conditions_display extends observer_interface;
    protected real temperature;
    protected real humidity;
    protected real pressure;
    protected string display_name;
    
    function new(string name = "Current Conditions");
      display_name = name;
    endfunction
    
    virtual function void update(real temp, real humid, real press);
      temperature = temp;
      humidity = humid;
      pressure = press;
      display();
    endfunction
    
    function void display();
      $display("[%s] Current conditions:", display_name);
      $display("  Temperature: %0.1f C", temperature);
      $display("  Humidity: %0.1f%%", humidity);
      $display("  Pressure: %0.1f hPa", pressure);
    endfunction
  endclass
  
  // Concrete Observer - Statistics Display
  class statistics_display extends observer_interface;
    protected real temp_sum;
    protected real temp_min;
    protected real temp_max;
    protected int measurement_count;
    protected string display_name;
    
    function new(string name = "Statistics");
      display_name = name;
      temp_sum = 0.0;
      temp_min = 999.0;
      temp_max = -999.0;
      measurement_count = 0;
    endfunction
    
    virtual function void update(real temp, real humid, real press);
      temp_sum += temp;
      measurement_count++;
      
      if (temp < temp_min) temp_min = temp;
      if (temp > temp_max) temp_max = temp;
      
      display();
    endfunction
    
    function void display();
      real avg_temp = temp_sum / measurement_count;
      $display("[%s] Temperature statistics:", display_name);
      $display("  Average: %0.1f C", avg_temp);
      $display("  Min: %0.1f C", temp_min);
      $display("  Max: %0.1f C", temp_max);
      $display("  Measurements: %0d", measurement_count);
    endfunction
  endclass
  
  // Concrete Observer - Forecast Display
  class forecast_display extends observer_interface;
    protected real last_pressure;
    protected string display_name;
    
    function new(string name = "Forecast");
      display_name = name;
      last_pressure = 0.0;
    endfunction
    
    virtual function void update(real temp, real humid, real press);
      string forecast;
      
      if (last_pressure == 0.0) begin
        forecast = "No trend data available";
      end else if (press > last_pressure) begin
        forecast = "Improving weather on the way!";
      end else if (press < last_pressure) begin
        forecast = "Watch out for cooler, rainy weather";
      end else begin
        forecast = "More of the same";
      end
      
      last_pressure = press;
      display(forecast);
    endfunction
    
    function void display(string forecast);
      $display("[%s] Weather forecast:", display_name);
      $display("  %s", forecast);
    endfunction
  endclass

endpackage

// Top-level design module
module weather_station_observer;
  import weather_observer_pkg::*;
  
  // Declare objects
  weather_station station;
  current_conditions_display current_display;
  statistics_display stats_display;
  forecast_display forecast_display_obj;
  
  initial begin
    $display("=== Basic Observer Pattern Example ===");
    $display();
    
    // Create the weather station (subject)
    station = new();
    
    // Create observers
    current_display = new("Current Conditions Display");
    stats_display = new("Statistics Display");
    forecast_display_obj = new("Forecast Display");
    
    // Register observers with the subject
    $display("--- Registering Observers ---");
    station.attach_observer(current_display);
    station.attach_observer(stats_display);
    station.attach_observer(forecast_display_obj);
    $display();
    
    // Simulate weather measurements
    $display("--- First Weather Update ---");
    station.set_measurements(25.5, 65.0, 1013.2);
    $display();
    
    $display("--- Second Weather Update ---");
    station.set_measurements(27.3, 70.0, 1015.8);
    $display();
    
    $display("--- Third Weather Update ---");
    station.set_measurements(23.1, 60.0, 1010.5);
    $display();
    
    // Demonstrate observer removal
    $display("--- Removing Statistics Display ---");
    station.detach_observer(stats_display);
    $display();
    
    $display("--- Fourth Weather Update (fewer observers) ---");
    station.set_measurements(26.0, 68.0, 1012.0);
    $display();
    
    $display("=== Observer Pattern Example Complete ===");
  end

endmodule
```

```systemverilog
// weather_station_observer_testbench.sv
module weather_station_testbench;
  
  // Instantiate the design under test
  weather_station_observer WEATHER_OBSERVER_INSTANCE();
  
  initial begin
    // Dump waves for debugging
    $dumpfile("weather_station_testbench.vcd");
    $dumpvars(0, weather_station_testbench);
    
    // Wait for design to complete
    #100;
    
    $display("Hello from weather station observer testbench!");
    $display();
    
    // Simulation complete
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Basic Observer Pattern Example ===

--- Registering Observers ---
[SUBJECT] Observer attached. Total observers: 1
[SUBJECT] Observer attached. Total observers: 2
[SUBJECT] Observer attached. Total observers: 3

--- First Weather Update ---
[WEATHER_STATION] New measurements recorded:
  Temperature: 25.5 C
  Humidity: 65.0%
  Pressure: 1013.2 hPa
[WEATHER_STATION] Notifying 3 observers...
[Current Conditions Display] Current conditions:
  Temperature: 25.5 C
  Humidity: 65.0%
  Pressure: 1013.2 hPa
[Statistics Display] Temperature statistics:
  Average: 25.5 C
  Min: 25.5 C
  Max: 25.5 C
  Measurements: 1
[Forecast Display] Weather forecast:
  No trend data available

--- Second Weather Update ---
[WEATHER_STATION] New measurements recorded:
  Temperature: 27.3 C
  Humidity: 70.0%
  Pressure: 1015.8 hPa
[WEATHER_STATION] Notifying 3 observers...
[Current Conditions Display] Current conditions:
  Temperature: 27.3 C
  Humidity: 70.0%
  Pressure: 1015.8 hP

0

### **Example 23:** Simple Strategy Pattern
**Description:** SortStrategy base class with different sorting algorithms. Context class uses different strategies polymorphically.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_23__simple_strategy_pattern/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// sorting_strategy_pattern.sv
package sorting_strategy_pkg;

  // Strategy interface - defines common interface for all sorting strategies
  virtual class SortStrategy;
    pure virtual function void sort(ref int data[]);
    pure virtual function string get_name();
  endclass

  // Concrete strategy 1: Bubble Sort
  class BubbleSortStrategy extends SortStrategy;
    
    virtual function void sort(ref int data[]);
      int temp;
      int n = data.size();
      
      for (int i = 0; i < n - 1; i++) begin
        for (int j = 0; j < n - i - 1; j++) begin
          if (data[j] > data[j + 1]) begin
            temp = data[j];
            data[j] = data[j + 1];
            data[j + 1] = temp;
          end
        end
      end
    endfunction
    
    virtual function string get_name();
      return "Bubble Sort";
    endfunction
    
  endclass

  // Concrete strategy 2: Selection Sort
  class SelectionSortStrategy extends SortStrategy;
    
    virtual function void sort(ref int data[]);
      int min_idx, temp;
      int n = data.size();
      
      for (int i = 0; i < n - 1; i++) begin
        min_idx = i;
        for (int j = i + 1; j < n; j++) begin
          if (data[j] < data[min_idx]) begin
            min_idx = j;
          end
        end
        if (min_idx != i) begin
          temp = data[i];
          data[i] = data[min_idx];
          data[min_idx] = temp;
        end
      end
    endfunction
    
    virtual function string get_name();
      return "Selection Sort";
    endfunction
    
  endclass

  // Concrete strategy 3: Insertion Sort
  class InsertionSortStrategy extends SortStrategy;
    
    virtual function void sort(ref int data[]);
      int key, j;
      int n = data.size();
      
      for (int i = 1; i < n; i++) begin
        key = data[i];
        j = i - 1;
        while (j >= 0 && data[j] > key) begin
          data[j + 1] = data[j];
          j = j - 1;
        end
        data[j + 1] = key;
      end
    endfunction
    
    virtual function string get_name();
      return "Insertion Sort";
    endfunction
    
  endclass

  // Context class that uses different sorting strategies
  class SortingContext;
    SortStrategy strategy;
    
    function new(SortStrategy initial_strategy);
      this.strategy = initial_strategy;
    endfunction
    
    function void set_strategy(SortStrategy new_strategy);
      this.strategy = new_strategy;
    endfunction
    
    function void execute_sort(ref int data[]);
      if (strategy != null) begin
        $display("Using strategy: %s", strategy.get_name());
        strategy.sort(data);
      end else begin
        $display("Error: No sorting strategy set!");
      end
    endfunction
    
    function string get_current_strategy_name();
      if (strategy != null) begin
        return strategy.get_name();
      end else begin
        return "No strategy set";
      end
    endfunction
    
  endclass

endpackage

// Main design module
module sorting_strategy_pattern;
  import sorting_strategy_pkg::*;
  
  initial begin
    $display("=== Strategy Pattern Example: Sorting Algorithms ===");
    $display();
  end
  
endmodule
```

```systemverilog
// sorting_strategy_pattern_testbench.sv
module sorting_strategy_testbench;
  import sorting_strategy_pkg::*;
  
  // Instantiate design under test
  sorting_strategy_pattern DESIGN_INSTANCE();
  
  // Test data arrays
  int test_data1[] = '{64, 34, 25, 12, 22, 11, 90};
  int test_data2[] = '{64, 34, 25, 12, 22, 11, 90};
  int test_data3[] = '{64, 34, 25, 12, 22, 11, 90};
  int reference_data[] = '{64, 34, 25, 12, 22, 11, 90};
  
  // Strategy objects
  BubbleSortStrategy bubble_strategy;
  SelectionSortStrategy selection_strategy;
  InsertionSortStrategy insertion_strategy;
  SortingContext sort_context;
  
  initial begin
    // Initialize VCD dumping
    $dumpfile("sorting_strategy_testbench.vcd");
    $dumpvars(0, sorting_strategy_testbench);
    
    // Create strategy objects
    bubble_strategy = new();
    selection_strategy = new();
    insertion_strategy = new();
    
    // Create context with initial strategy
    sort_context = new(bubble_strategy);
    
    $display("=== Testing Strategy Pattern with Sorting Algorithms ===");
    $display();
    
    // Display original data
    $display("Original data: %p", reference_data);
    $display();
    
    // Test 1: Bubble Sort Strategy
    $display("--- Test 1: Bubble Sort Strategy ---");
    $display("Current strategy: %s", sort_context.get_current_strategy_name());
    display_array("Before sorting", test_data1);
    sort_context.execute_sort(test_data1);
    display_array("After sorting", test_data1);
    verify_sorted(test_data1, "Bubble Sort");
    $display();
    
    // Test 2: Selection Sort Strategy
    $display("--- Test 2: Selection Sort Strategy ---");
    sort_context.set_strategy(selection_strategy);
    $display("Current strategy: %s", sort_context.get_current_strategy_name());
    display_array("Before sorting", test_data2);
    sort_context.execute_sort(test_data2);
    display_array("After sorting", test_data2);
    verify_sorted(test_data2, "Selection Sort");
    $display();
    
    // Test 3: Insertion Sort Strategy
    $display("--- Test 3: Insertion Sort Strategy ---");
    sort_context.set_strategy(insertion_strategy);
    $display("Current strategy: %s", sort_context.get_current_strategy_name());
    display_array("Before sorting", test_data3);
    sort_context.execute_sort(test_data3);
    display_array("After sorting", test_data3);
    verify_sorted(test_data3, "Insertion Sort");
    $display();
    
    // Test 4: Strategy switching demonstration
    $display("--- Test 4: Strategy Switching Demonstration ---");
    demonstrate_strategy_switching();
    
    $display("=== Strategy Pattern Test Complete ===");
    $display();
    
    // Wait and finish
    #10;
    $finish;
  end
  
  // Helper function to display array contents
  function void display_array(string label, int data[]);
    $write("%s: [", label);
    for (int i = 0; i < data.size(); i++) begin
      if (i > 0) $write(", ");
      $write("%0d", data[i]);
    end
    $display("]");
  endfunction
  
  // Helper function to verify array is sorted
  function automatic void verify_sorted(int data[], string strategy_name);
    bit is_sorted = 1;
    for (int i = 0; i < data.size() - 1; i++) begin
      if (data[i] > data[i + 1]) begin
        is_sorted = 0;
        break;
      end
    end
    
    if (is_sorted) begin
      $display("âœ“ %s: Array is correctly sorted!", strategy_name);
    end else begin
      $display("âœ— %s: Array is NOT sorted correctly!", strategy_name);
    end
  endfunction
  
  // Demonstrate dynamic strategy switching
  function automatic void demonstrate_strategy_switching();
    int demo_data[];
    int temp_data[];
    
    // Initialize arrays
    demo_data = new[6];
    demo_data = '{5, 2, 8, 1, 9, 3};
    
    $display("Demonstrating dynamic strategy switching:");
    $display("Original data: %p", demo_data);
    
    // Test with Bubble Sort
    temp_data = new[6];
    temp_data = '{5, 2, 8, 1, 9, 3};
    sort_context.set_strategy(bubble_strategy);
    $display("Switched to: Bubble Sort");
    sort_context.execute_sort(temp_data);
    $display("Result: %p", temp_data);
    $display();
    
    // Test with Selection Sort
    temp_data = '{5, 2, 8, 1, 9, 3};
    sort_context.set_strategy(selection_strategy);
    $display("Switched to: Selection Sort");
    sort_context.execute_sort(temp_data);
    $display("Result: %p", temp_data);
    $display();
    
    // Test with Insertion Sort
    temp_data = '{5, 2, 8, 1, 9, 3};
    sort_context.set_strategy(insertion_strategy);
    $display("Switched to: Insertion Sort");
    sort_context.execute_sort(temp_data);
    $display("Result: %p", temp_data);
    $display();
  endfunction
  
endmodule
```

Verilator Simulation Output:
=== Strategy Pattern Example: Sorting Algorithms ===

=== Testing Strategy Pattern with Sorting Algorithms ===

Original data: '{'h40, 'h22, 'h19, 'hc, 'h16, 'hb, 'h5a}

--- Test 1: Bubble Sort Strategy ---
Current strategy: Bubble Sort
Before sorting: [64, 34, 25, 12, 22, 11, 90]
Using strategy: Bubble Sort
After sorting: [11, 12, 22, 25, 34, 64, 90]
âœ“ Bubble Sort: Array is correctly sorted!

--- Test 2: Selection Sort Strategy ---
Current strategy: Selection Sort
Before sorting: [64, 34, 25, 12, 22, 11, 90]
Using strategy: Selection Sort
After sorting: [11, 12, 22, 25, 34, 64, 90]
âœ“ Selection Sort: Array is correctly sorted!

--- Test 3: Insertion Sort Strategy ---
Current strategy: Insertion Sort
Before sorting: [64, 34, 25, 12, 22, 11, 90]
Using strategy: Insertion Sort
After sorting: [11, 12, 22, 25, 34, 64, 90]
âœ“ Insertion Sort: Array is correctly sorted!

--- Test 4: Strategy Switching Demonstration ---
Demonstrating dynamic strategy switching:

0

## Common Pitfalls and Debugging Tips

### Forgetting virtual keyword
```systemverilog
// Wrong: Method won't be overridden properly
function void my_method();  // Not virtual

// Correct: Use virtual for overrideable methods
virtual function void my_method();
```

### Incorrect super usage
```systemverilog
// Wrong: Calling super incorrectly
function new();
    super();  // Syntax error

// Correct: Proper super call
function new();
    super.new();  // Correct syntax
```

### Handle assignment vs object copying
```systemverilog
// This copies the handle, not the object
Shape shape1 = new Circle(5.0);
Shape shape2 = shape1;  // Both handles point to same object
```

### **Example 24** Virtual Keyword Demo
**Description:** Side-by-side comparison showing difference between virtual and non-virtual methods in inheritance hierarchy.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_24__virtual_keyword_demo/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// virtual_keyword_demo.sv - Virtual keyword demonstration
package virtual_keyword_pkg;

  // Base class with both virtual and non-virtual methods
  class base_shape_class;
    string shape_type;
    
    function new(string name = "base_shape");
      shape_type = name;
    endfunction
    
    // Non-virtual method - will NOT be overridden in polymorphic calls
    function void display_info();
      $display("Base: This is a %s shape", shape_type);
    endfunction
    
    // Virtual method - CAN be overridden in polymorphic calls
    virtual function void draw_shape();
      $display("Base: Drawing generic shape");
    endfunction
    
    // Virtual method for calculating area
    virtual function real get_area();
      $display("Base: Generic area calculation");
      return 0.0;
    endfunction
    
    // Non-virtual method for identification
    function string get_class_name();
      return "base_shape_class";
    endfunction
  endclass

  // Derived class - Circle
  class circle_shape_class extends base_shape_class;
    real radius;
    
    function new(real r = 1.0);
      super.new("circle");
      radius = r;
    endfunction
    
    // Override non-virtual method (will only work with direct calls)
    function void display_info();
      $display("Circle: This is a %s with radius %.2f", shape_type, radius);
    endfunction
    
    // Override virtual method (will work with polymorphic calls)
    virtual function void draw_shape();
      $display("Circle: Drawing circle with radius %.2f", radius);
    endfunction
    
    // Override virtual method for area calculation
    virtual function real get_area();
      real area = 3.14159 * radius * radius;
      $display("Circle: Area = %.2f", area);
      return area;
    endfunction
    
    // Override non-virtual method
    function string get_class_name();
      return "circle_shape_class";
    endfunction
  endclass

  // Derived class - Rectangle
  class rectangle_shape_class extends base_shape_class;
    real width, height;
    
    function new(real w = 1.0, real h = 1.0);
      super.new("rectangle");
      width = w;
      height = h;
    endfunction
    
    // Override non-virtual method
    function void display_info();
      $display("Rectangle: This is a %s %.2fx%.2f", shape_type, width, height);
    endfunction
    
    // Override virtual method
    virtual function void draw_shape();
      $display("Rectangle: Drawing rectangle %.2fx%.2f", width, height);
    endfunction
    
    // Override virtual method for area calculation
    virtual function real get_area();
      real area = width * height;
      $display("Rectangle: Area = %.2f", area);
      return area;
    endfunction
    
    // Override non-virtual method
    function string get_class_name();
      return "rectangle_shape_class";
    endfunction
  endclass

  // Demonstration class to show polymorphism
  class shape_manager_class;
    base_shape_class shape_array[];
    circle_shape_class temp_circle;
    rectangle_shape_class temp_rect;
    
    function new();
      shape_array = new[3];
      shape_array[0] = new("generic");
      temp_circle = new(2.5);
      shape_array[1] = temp_circle;
      temp_rect = new(3.0, 4.0);
      shape_array[2] = temp_rect;
    endfunction
    
    // Demonstrate polymorphic behavior
    function void demonstrate_polymorphism();
      $display("\n=== POLYMORPHIC CALLS (using base class handle) ===");
      
      foreach(shape_array[i]) begin
        $display("\nShape %0d:", i);
        
        // Non-virtual method - always calls base class version
        $display("Non-virtual display_info():");
        shape_array[i].display_info();
        
        // Virtual method - calls appropriate derived class version
        $display("Virtual draw_shape():");
        shape_array[i].draw_shape();
        
        // Virtual method - calls appropriate derived class version
        $display("Virtual get_area():");
        void'(shape_array[i].get_area());
        
        // Non-virtual method - always calls base class version
        $display("Non-virtual get_class_name(): %s", 
                 shape_array[i].get_class_name());
      end
    endfunction
    
    // Demonstrate direct calls (non-polymorphic)
    function void demonstrate_direct_calls();
      circle_shape_class circle_handle;
      rectangle_shape_class rect_handle;
      
      $display("\n=== DIRECT CALLS (using specific class handles) ===");
      
      // Direct call to circle
      circle_handle = new(1.5);
      $display("\nDirect Circle calls:");
      circle_handle.display_info();        // Calls circle version
      circle_handle.draw_shape();          // Calls circle version
      void'(circle_handle.get_area());     // Calls circle version
      $display("Class name: %s", circle_handle.get_class_name());
      
      // Direct call to rectangle
      rect_handle = new(2.0, 3.5);
      $display("\nDirect Rectangle calls:");
      rect_handle.display_info();          // Calls rectangle version
      rect_handle.draw_shape();            // Calls rectangle version
      void'(rect_handle.get_area());       // Calls rectangle version
      $display("Class name: %s", rect_handle.get_class_name());
    endfunction
    
    // Show the difference clearly
    function void show_virtual_vs_nonvirtual();
      circle_shape_class specific_circle;
      base_shape_class generic_handle;
      
      $display("\n=== VIRTUAL vs NON-VIRTUAL COMPARISON ===");
      
      specific_circle = new(3.0);
      generic_handle = specific_circle;  // Polymorphic assignment
      
      $display("\nUsing specific circle handle:");
      specific_circle.display_info();    // Circle version
      specific_circle.draw_shape();      // Circle version
      
      $display("\nUsing base class handle (polymorphic):");
      generic_handle.display_info();     // Base version (non-virtual!)
      generic_handle.draw_shape();       // Circle version (virtual!)
      
      $display("\nKey difference:");
      $display("- Non-virtual methods use the handle's declared type");
      $display("- Virtual methods use the object's actual type");
    endfunction
  endclass

endpackage

// Design module
module virtual_keyword_demo_module;
  import virtual_keyword_pkg::*;
  
  shape_manager_class manager;
  
  initial begin
    $display("=== Virtual Keyword Demonstration ===");
    
    manager = new();
    
    // Show polymorphic behavior
    manager.demonstrate_polymorphism();
    
    // Show direct calls
    manager.demonstrate_direct_calls();
    
    // Show clear comparison
    manager.show_virtual_vs_nonvirtual();
    
    $display("\n=== Summary ===");
    $display("Virtual methods enable true polymorphism in SystemVerilog");
    $display("Non-virtual methods are resolved at compile time");
    $display("Virtual methods are resolved at runtime");
  end
endmodule
```

```systemverilog
// virtual_keyword_demo_testbench.sv
module virtual_keyword_testbench;
  import virtual_keyword_pkg::*;
  
  // Instantiate design under test
  virtual_keyword_demo_module DESIGN_INSTANCE();
  
  // Additional test scenarios
  base_shape_class shape_handle;
  circle_shape_class circle_obj;
  rectangle_shape_class rect_obj;
  
  initial begin
    // Setup for VCD dumping
    $dumpfile("virtual_keyword_testbench.vcd");
    $dumpvars(0, virtual_keyword_testbench);
    
    #1; // Wait for design to complete
    
    $display("\n============================================================");
    $display("TESTBENCH: Additional Virtual Keyword Tests");
    $display("============================================================");
    
    // Test 1: Dynamic type checking scenario
    test_dynamic_behavior();
    
    // Test 2: Method resolution demonstration
    test_method_resolution();
    
    // Test 3: Inheritance chain demonstration
    test_inheritance_chain();
    
    $display("\n============================================================");
    $display("TESTBENCH: All tests completed successfully!");
    $display("============================================================");
    
    #5; // Final wait
    $finish;
  end
  
  // Test dynamic behavior with different object types
  task test_dynamic_behavior();
    $display("\n--- Test 1: Dynamic Behavior ---");
    
    // Create different objects but use same handle type
    shape_handle = new();
    shape_handle.shape_type = "base";
    call_methods("Base object");
    
    circle_obj = new(2.0);
    shape_handle = circle_obj;
    call_methods("Circle object via base handle");
    
    rect_obj = new(1.5, 2.5);
    shape_handle = rect_obj;
    call_methods("Rectangle object via base handle");
  endtask
  
  // Helper task to call methods and show differences
  task call_methods(string description);
    $display("\n%s:", description);
    $display("  Non-virtual display_info(): ", $sformatf(""));
    shape_handle.display_info();
    $display("  Virtual draw_shape(): ", $sformatf(""));
    shape_handle.draw_shape();
    $display("  Virtual get_area(): %.2f", shape_handle.get_area());
    $display("  Non-virtual get_class_name(): %s", 
             shape_handle.get_class_name());
  endtask
  
  // Demonstrate method resolution rules
  task test_method_resolution();
    $display("\n--- Test 2: Method Resolution Rules ---");
    
    circle_obj = new(1.0);
    
    $display("\nDirect call (circle_obj.method()):");
    circle_obj.display_info();
    circle_obj.draw_shape();
    
    $display("\nPolymorphic call (base_handle.method()):");
    shape_handle = circle_obj;
    shape_handle.display_info();  // Uses base class (non-virtual)
    shape_handle.draw_shape();    // Uses circle class (virtual)
    
    $display("\nExplanation:");
    $display("- display_info() is non-virtual: uses handle's type");
    $display("- draw_shape() is virtual: uses object's actual type");
  endtask
  
  // Show inheritance method resolution
  task test_inheritance_chain();
    base_shape_class shapes[];
    circle_shape_class temp_circle;
    rectangle_shape_class temp_rect;
    
    $display("\n--- Test 3: Inheritance Chain ---");
    
    shapes = new[4];
    shapes[0] = new();
    shapes[0].shape_type = "base";
    
    temp_circle = new(1.0);
    shapes[1] = temp_circle;
    
    temp_rect = new(2.0, 2.0);
    shapes[2] = temp_rect;
    
    temp_circle = new(3.0);
    shapes[3] = temp_circle;
    
    $display("\nProcessing array of shapes:");
    foreach(shapes[i]) begin
      $display("\nShape[%0d] processing:", i);
      shapes[i].draw_shape();        // Virtual - calls correct version
      void'(shapes[i].get_area());   // Virtual - calls correct version
      $display("Handle type reports: %s", shapes[i].get_class_name());
    end
    
    $display("\nNote: All objects called through base_shape_class handle");
    $display("Virtual methods found correct implementation");
    $display("Non-virtual method always used base class version");
  endtask
  
endmodule
```

Verilator Simulation Output:
=== Virtual Keyword Demonstration ===

=== POLYMORPHIC CALLS (using base class handle) ===

Shape 0:
Non-virtual display_info():
Base: This is a generic shape
Virtual draw_shape():
Base: Drawing generic shape
Virtual get_area():
Base: Generic area calculation
Non-virtual get_class_name(): base_shape_class

Shape 1:
Non-virtual display_info():
Base: This is a circle shape
Virtual draw_shape():
Circle: Drawing circle with radius 2.50
Virtual get_area():
Circle: Area = 19.63
Non-virtual get_class_name(): base_shape_class

Shape 2:
Non-virtual display_info():
Base: This is a rectangle shape
Virtual draw_shape():
Rectangle: Drawing rectangle 3.00x4.00
Virtual get_area():
Rectangle: Area = 12.00
Non-virtual get_class_name(): base_shape_class

=== DIRECT CALLS (using specific class handles) ===

Direct Circle calls:
Circle: This is a circle with radius 1.50
Circle: Drawing circle with radius 1.50
Circle: Area = 7.07
Class name: circle_shape_class

Direct Rectangle

0

### **Example 25:** Handle vs Object Copying
**Description:** Simple example demonstrating the difference between copying object handles and actual object duplication.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_25__handle_vs_object_copying/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// handle_vs_object_copying.sv
// Demonstrates the difference between handle copying and object copying

package vehicle_package;

  // Simple vehicle class to demonstrate copying concepts
  class vehicle_class;
    string brand;
    int speed;
    
    function new(string init_brand = "Unknown", int init_speed = 0);
      this.brand = init_brand;
      this.speed = init_speed;
    endfunction
    
    function void display(string prefix = "");
      $display("%sVehicle: %s, Speed: %0d", prefix, brand, speed);
    endfunction
    
    function void set_speed(int new_speed);
      this.speed = new_speed;
    endfunction
    
    function void set_brand(string new_brand);
      this.brand = new_brand;
    endfunction
    
    // Deep copy method - creates a new object with same values
    function vehicle_class deep_copy();
      vehicle_class new_vehicle;
      new_vehicle = new(this.brand, this.speed);
      return new_vehicle;
    endfunction
    
  endclass

endpackage

module handle_vs_object_copying_module;
  
  import vehicle_package::*;
  
  initial begin
    vehicle_class original_vehicle;
    vehicle_class handle_copy;
    vehicle_class object_copy;
    
    $display("=== Handle vs Object Copying Example ===");
    $display();
    
    // Create original vehicle
    original_vehicle = new("Toyota", 60);
    $display("1. Created original vehicle:");
    original_vehicle.display("   ");
    $display();
    
    // Handle copying - both variables point to same object
    $display("2. Handle copying (shallow copy):");
    handle_copy = original_vehicle;  // Just copying the handle/reference
    $display("   Original vehicle:");
    original_vehicle.display("   ");
    $display("   Handle copy:");
    handle_copy.display("   ");
    $display();
    
    // Modify through handle copy
    $display("3. Modifying speed through handle copy:");
    handle_copy.set_speed(80);
    $display("   Original vehicle (affected!):");
    original_vehicle.display("   ");
    $display("   Handle copy:");
    handle_copy.display("   ");
    $display("   -> Both changed because they reference same object");
    $display();
    
    // Object copying - creates independent copy
    $display("4. Object copying (deep copy):");
    object_copy = original_vehicle.deep_copy();
    $display("   Original vehicle:");
    original_vehicle.display("   ");
    $display("   Object copy:");
    object_copy.display("   ");
    $display();
    
    // Modify original after object copy
    $display("5. Modifying original after object copy:");
    original_vehicle.set_brand("Honda");
    original_vehicle.set_speed(100);
    $display("   Original vehicle (modified):");
    original_vehicle.display("   ");
    $display("   Handle copy (also modified - same object!):");
    handle_copy.display("   ");
    $display("   Object copy (unchanged - independent object!):");
    object_copy.display("   ");
    $display();
    
    // Modify object copy
    $display("6. Modifying object copy:");
    object_copy.set_brand("Ford");
    object_copy.set_speed(45);
    $display("   Original vehicle (unchanged):");
    original_vehicle.display("   ");
    $display("   Object copy (modified independently):");
    object_copy.display("   ");
    $display();
    
    $display("=== Key Takeaways ===");
    $display("- Handle copying: Creates another reference to same object");
    $display("- Object copying: Creates independent object with same values");
    $display("- Changes through handle copies affect the original object");
    $display("- Changes to deep copies don't affect the original object");
    
  end

endmodule
```

```systemverilog
// handle_vs_object_copying_testbench.sv
// Testbench for handle vs object copying demonstration

module handle_vs_object_copying_testbench;

  // Instantiate the design module
  handle_vs_object_copying_module DESIGN_INSTANCE();

  initial begin
    // Dump waves for verilator
    $dumpfile("handle_vs_object_copying_testbench.vcd");
    $dumpvars(0, handle_vs_object_copying_testbench);
    
    // Wait for design to complete
    #10;
    
    $display();
    $display("=== Testbench Complete ===");
    $display("Handle vs Object Copying example finished successfully!");
    
    // Finish simulation
    $finish;
  end

endmodule
```

Verilator Simulation Output:
=== Handle vs Object Copying Example ===

1. Created original vehicle:
   Vehicle: Toyota, Speed: 60

2. Handle copying (shallow copy):
   Original vehicle:
   Vehicle: Toyota, Speed: 60
   Handle copy:
   Vehicle: Toyota, Speed: 60

3. Modifying speed through handle copy:
   Original vehicle (affected!):
   Vehicle: Toyota, Speed: 80
   Handle copy:
   Vehicle: Toyota, Speed: 80
   -> Both changed because they reference same object

4. Object copying (deep copy):
   Original vehicle:
   Vehicle: Toyota, Speed: 80
   Object copy:
   Vehicle: Toyota, Speed: 80

5. Modifying original after object copy:
   Original vehicle (modified):
   Vehicle: Honda, Speed: 100
   Handle copy (also modified - same object!):
   Vehicle: Honda, Speed: 100
   Object copy (unchanged - independent object!):
   Vehicle: Toyota, Speed: 80

6. Modifying object copy:
   Original vehicle (unchanged):
   Vehicle: Honda, Speed: 100
   Object copy (modified independently):
   Vehicle: F

0

### **Example 26:** Proper super Usage
**Description:** Shows correct and incorrect ways to call parent constructors and methods using the super keyword.

In [ ]:
# | echo: false
from read_files_utils import read_sv_files
from verilator_runner import run_docker_compose

files_path = "Chapter_10_examples/example_26__proper_super_usage/"

read_sv_files(files_path)

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// super_usage_example.sv
// Chapter 10 Example 26: Proper super Usage

package super_usage_pkg;

  // Base class - Animal
  class Animal;
    protected string name;
    protected int age;
    
    // Constructor
    function new(string animal_name = "Unknown", int animal_age = 0);
      name = animal_name;
      age = animal_age;
      $display("Animal constructor: %s, age %0d", name, age);
    endfunction
    
    // Virtual method for making sound
    virtual function string make_sound();
      return "Generic animal sound";
    endfunction
    
    // Method to display animal info
    virtual function void display_info();
      $display("Animal: %s, Age: %0d", name, age);
    endfunction
    
    // Method to celebrate birthday
    virtual function void birthday();
      age++;
      $display("%s is now %0d years old!", name, age);
    endfunction
  endclass

  // Derived class - Dog
  class Dog extends Animal;
    protected string breed;
    
    // Constructor - CORRECT super usage
    function new(string dog_name = "Buddy", int dog_age = 1, 
                 string dog_breed = "Mixed");
      super.new(dog_name, dog_age);  // Call parent constructor FIRST
      breed = dog_breed;
      $display("Dog constructor: %s is a %s", dog_name, breed);
    endfunction
    
    // Override make_sound - CORRECT super usage
    function string make_sound();
      string parent_sound = super.make_sound();  // Call parent method
      return $sformatf("%s -> Woof! Woof!", parent_sound);
    endfunction
    
    // Override display_info - CORRECT super usage
    function void display_info();
      super.display_info();  // Call parent method first
      $display("Breed: %s", breed);
    endfunction
    
    // Override birthday - CORRECT super usage
    function void birthday();
      $display("It's %s's birthday!", name);
      super.birthday();  // Call parent method
      $display("%s the %s is wagging tail!", name, breed);
    endfunction
    
    // Dog-specific method
    function void fetch();
      $display("%s is fetching the ball!", name);
    endfunction
  endclass

  // Derived class - Cat (shows different super usage patterns)
  class Cat extends Animal;
    protected bit is_indoor;
    
    // Constructor - CORRECT super usage with default parameters
    function new(string cat_name = "Whiskers", int cat_age = 2, 
                 bit indoor = 1);
      super.new(cat_name, cat_age);  // Always call super.new() first
      is_indoor = indoor;
      $display("Cat constructor: %s, indoor: %s", cat_name, 
               is_indoor ? "Yes" : "No");
    endfunction
    
    // Override make_sound - CORRECT super usage in conditional
    function string make_sound();
      if (age < 1) begin
        return "Mew mew";  // Kittens don't call parent
      end else begin
        string parent_sound = super.make_sound();
        return $sformatf("%s -> Meow! Purr!", parent_sound);
      end
    endfunction
    
    // Override display_info - CORRECT super usage with additions
    function void display_info();
      super.display_info();  // Call parent method
      $display("Indoor cat: %s", is_indoor ? "Yes" : "No");
    endfunction
    
    // Cat-specific method
    function void climb();
      $display("%s is climbing the cat tree!", name);
    endfunction
  endclass

  // Example of INCORRECT super usage (commented out for reference)
  /*
  class BadDog extends Animal;
    function new(string name = "BadDog");
      // WRONG: Setting attributes before calling super.new()
      this.name = name;
      super.new(name, 1);  // This could cause issues
    endfunction
    
    function string make_sound();
      // WRONG: Not storing super result properly
      super.make_sound();  // Return value ignored
      return "Woof!";
    endfunction
    
    function void display_info();
      // WRONG: Not calling super at all
      $display("Bad dog: %s", name);  // Missing parent info
    endfunction
  endclass
  */

endpackage
```

```systemverilog
// super_usage_example_testbench.sv
// Chapter 10 Example 26: Proper super Usage Testbench

module super_usage_testbench;
  import super_usage_pkg::*;
  
  initial begin
    // Dump waves for debugging
    $dumpfile("super_usage_testbench.vcd");
    $dumpvars(0, super_usage_testbench);
    
    $display("=== Chapter 10 Example 26: Proper super Usage ===");
    $display();
    
    // Test 1: Create base Animal
    $display("--- Test 1: Base Animal Creation ---");
    begin
      Animal base_animal = new("Generic Animal", 5);
      base_animal.display_info();
      $display("Sound: %s", base_animal.make_sound());
      $display();
    end
    
    // Test 2: Create Dog (demonstrates proper super usage)
    $display("--- Test 2: Dog Creation (Proper super usage) ---");
    begin
      Dog my_dog = new("Rex", 3, "Golden Retriever");
      my_dog.display_info();
      $display("Sound: %s", my_dog.make_sound());
      my_dog.fetch();
      $display();
    end
    
    // Test 3: Create Cat (demonstrates different super patterns)
    $display("--- Test 3: Cat Creation (Different super patterns) ---");
    begin
      Cat my_cat = new("Luna", 2, 1);
      my_cat.display_info();
      $display("Sound: %s", my_cat.make_sound());
      my_cat.climb();
      $display();
    end
    
    // Test 4: Polymorphism with proper super usage
    $display("--- Test 4: Polymorphism Test ---");
    begin
      Animal wild_animal = new("Wild Animal", 10);
      $display("Animal 0:");
      wild_animal.display_info();
      $display("Sound: %s", wild_animal.make_sound());
      $display();
    end
    
    begin
      Dog buddy_dog = new("Buddy", 4, "Labrador");
      $display("Animal 1:");
      buddy_dog.display_info();
      $display("Sound: %s", buddy_dog.make_sound());
      $display();
    end
    
    begin
      Cat mittens_cat = new("Mittens", 1, 0);
      $display("Animal 2:");
      mittens_cat.display_info();
      $display("Sound: %s", mittens_cat.make_sound());
      $display();
    end
    
    // Test 5: Birthday method (shows super usage in overridden methods)
    $display("--- Test 5: Birthday Celebration (super in methods) ---");
    begin
      Animal base_animal = new("Birthday Animal", 4);
      $display("Base animal birthday:");
      base_animal.birthday();
      $display();
    end
    
    begin
      Dog birthday_dog = new("Birthday Dog", 2, "Poodle");
      $display("Dog birthday:");
      birthday_dog.birthday();
      $display();
    end
    
    begin
      Cat birthday_cat = new("Birthday Cat", 3, 1);
      $display("Cat birthday:");
      birthday_cat.birthday();
      $display();
    end
    
    // Test 6: Demonstrate constructor chain
    $display("--- Test 6: Constructor Chain Demonstration ---");
    $display("Creating new dog to show constructor chain:");
    begin
      Dog another_dog = new("Max", 2, "Beagle");
      another_dog.display_info();
      $display();
    end
    
    // Test 7: Show method resolution through base class reference
    $display("--- Test 7: Method Resolution ---");
    $display("Calling methods through base class reference:");
    begin
      Dog test_dog = new("Polymorphic Dog", 1, "Shepherd");
      Animal dog_as_animal = test_dog;
      dog_as_animal.display_info();  // Calls derived version
      $display("Sound: %s", dog_as_animal.make_sound());  // Calls derived
      $display();
    end
    
    // Test 8: Different constructor patterns
    $display("--- Test 8: Constructor Patterns ---");
    begin
      Animal default_animal = new();
      Dog default_dog = new();
      Cat default_cat = new();
      
      $display("Default constructors:");
      default_animal.display_info();
      default_dog.display_info();
      default_cat.display_info();
      $display();
    end
    
    $display("=== Key Points about Proper super Usage ===");
    $display("1. Always call super.new() FIRST in derived constructors");
    $display("2. Use super.method() to call parent method implementations");
    $display("3. super can be used conditionally in method overrides");
    $display("4. Store super method return values when needed");
    $display("5. Call super methods before or after derived logic as needed");
    $display();
    
    $display("=== Simulation Complete ===");
    $finish;
  end
  
endmodule
```

Verilator Simulation Output:

--- Test 1: Base Animal Creation ---
Animal constructor: Generic Animal, age 5
Animal: Generic Animal, Age: 5
Sound: Generic animal sound

--- Test 2: Dog Creation (Proper super usage) ---
Animal constructor: Rex, age 3
Dog constructor: Rex is a Golden Retriever
Animal: Rex, Age: 3
Breed: Golden Retriever
Sound: Generic animal sound -> Woof! Woof!
Rex is fetching the ball!

--- Test 3: Cat Creation (Different super patterns) ---
Animal constructor: Luna, age 2
Cat constructor: Luna, indoor: Yes
Animal: Luna, Age: 2
Indoor cat: Yes
Sound: Generic animal sound -> Meow! Purr!
Luna is climbing the cat tree!

--- Test 4: Polymorphism Test ---
Animal constructor: Wild Animal, age 10
Animal 0:
Animal: Wild Animal, Age: 10
Sound: Generic animal sound

Animal constructor: Buddy, age 4
Dog constructor: Buddy is a Labrador
Animal 1:
Animal: Buddy, Age: 4
Breed: Labrador
Sound: Generic animal sound -> Woof! Woof!

Animal constructor: Mittens, age 1
Cat constructor: Mi

0

## Summary

This chapter covered the essential object-oriented programming concepts in SystemVerilog:

- **Inheritance**: Creating new classes based on existing ones using `extends`
- **Method Overriding**: Providing specific implementations in derived classes
- **super keyword**: Accessing parent class methods and properties
- **Virtual Methods**: Enabling runtime method resolution for polymorphism
- **Abstract Classes**: Creating base classes with pure virtual methods
- **Polymorphism**: Treating objects of different types uniformly through common interfaces

These concepts enable you to write more modular, maintainable, and extensible SystemVerilog code by leveraging the power of object-oriented design principles. Understanding inheritance and polymorphism is crucial for building complex verification environments and reusable code libraries.